In [143]:
# Import libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import spatial

# SAFEGRAPH DATA ANALYSIS

In [144]:
# Import patterns and places data from SafeGraph
filepath_safegraph_patterns = ".././02 Data/SafeGraph Data/Patterns/"
filepath_safegraph_places = ".././02 Data/SafeGraph Data/Places/"
safegraph_patterns_df = pd.read_csv(filepath_safegraph_patterns + 'tangible_table_patterns_df.csv')
safegraph_places_df = pd.read_csv(filepath_safegraph_places + 'tangible_table_places_df.csv')

In [145]:
# Select the columns of patterns and places that are of interest
patterns_df = safegraph_patterns_df[['placekey', 'raw_visit_counts', 'raw_visitor_counts', 'visits_by_day',
        'visits_by_each_hour', 'poi_cbg', 'visitor_home_cbgs', 'visitor_daytime_cbgs', 'distance_from_home', 
        'normalized_visits_by_state_scaling','date_range_start', 'date_range_end', 'median_dwell','bucketed_dwell_times' ]]
places_df=safegraph_places_df[['placekey','latitude','longitude','top_category','open_hours', 'location_name']]

In [146]:
# Merge patterns with places to have lat, lon, top category
merged_df=patterns_df.merge(places_df[['placekey','latitude','longitude', 'top_category', 'open_hours', 'location_name']], how='left', left_on='placekey', right_on='placekey')
merged_df.head()

,placekey,raw_visit_counts,raw_visitor_counts,visits_by_day,visits_by_each_hour,poi_cbg,visitor_home_cbgs,visitor_daytime_cbgs,distance_from_home,normalized_visits_by_state_scaling,date_range_start,date_range_end,median_dwell,bucketed_dwell_times,latitude,longitude,top_category,open_hours,location_name
0,232-222@62j-sk3-p9z,74.0,45.0,"[11, 4, 4, 7, 14, 10, 24]","[3,2,2,2,2,2,3,3,4,4,5,5,8,11,14,11,11,16,10,1...",250173504002,"{""250214562002"":4,""120210112022"":4,""1720100020...","{""250173504002"":4,""250173391004"":4,""2501735210...",8926.0,1650.73334865,2021-10-01T00:00:00-04:00,2021-11-01T00:00:00-04:00,42.0,"{""<5"":1,""5-10"":19,""11-20"":4,""21-60"":23,""61-120...",NaN,NaN,NaN,NaN,NaN
1,zzy-222@62j-sjv-85f,141.0,121.0,"[14, 9, 10, 18, 18, 43, 29]","[4,4,2,2,1,2,0,0,10,11,14,13,20,17,21,22,20,16...",250173537004,"{""250173537003"":11,""250173537005"":6,""250173538...","{""250173537003"":8,""250173537005"":7,""2502141780...",6643.0,3145.31624540,2021-10-01T00:00:00-04:00,2021-11-01T00:00:00-04:00,12.0,"{""<5"":3,""5-10"":56,""11-20"":38,""21-60"":28,""61-12...",42.373312,-71.119151,Restaurants and Other Eating Places,"{ ""Mon"": [[""5:00"", ""23:00""]], ""Tue"": [[""5:00"",...",Starbucks
2,zzw-224@62j-sj2-s5z,22.0,18.0,"[2, 1, 4, 3, 4, 6, 2]","[5,3,3,3,2,2,2,2,5,4,4,4,4,4,4,5,6,5,2,4,7,10,...",250173531011,"{""250010121011"":4,""250250104051"":4}","{""250010121011"":4,""340230051002"":4,""4400302090...",23581.0,490.75856311,2021-10-01T00:00:00-04:00,2021-11-01T00:00:00-04:00,153.0,"{""<5"":0,""5-10"":3,""11-20"":2,""21-60"":2,""61-120"":...",42.363572,-71.101336,Restaurants and Other Eating Places,"{ ""Mon"": [[""11:30"", ""22:00""]], ""Tue"": [], ""Wed...",Mary Chung Restaurant
3,22m-222@62j-sjv-bc5,67.0,63.0,"[7, 5, 10, 6, 11, 12, 16]","[5,2,1,1,3,1,4,2,2,3,9,11,9,10,12,13,5,6,13,11...",250173541004,"{""210150703112"":5,""361190055003"":4,""2501735370...","{""250250001001"":4,""340030292003"":4,""2502142230...",35596.0,1494.58289675,2021-10-01T00:00:00-04:00,2021-11-01T00:00:00-04:00,44.0,"{""<5"":0,""5-10"":6,""11-20"":6,""21-60"":28,""61-120""...",42.373153,-71.120120,"Museums, Historical Sites, and Similar Institu...",NaN,Harvard Square
4,257-222@62j-sj2-m6k,93.0,39.0,"[12, 7, 7, 18, 18, 20, 11]","[27,27,26,24,24,27,25,24,25,29,30,26,22,20,20,...",250173529001,"{""250214151023"":8,""250251707012"":6,""2501735360...","{""250092081014"":4,""340030481001"":4,""2500920660...",6867.0,2074.57028952,2021-10-01T00:00:00-04:00,2021-11-01T00:00:00-04:00,88.0,"{""<5"":2,""5-10"":13,""11-20"":5,""21-60"":23,""61-120...",42.371619,-71.099913,Automobile Dealers,"{ ""Mon"": [[""8:00"", ""18:00""]], ""Tue"": [[""8:00"",...",Prospect Auto Inc


In [147]:
#Keep points of interest related to food
topcategory_list =['Drinking Places (Alcoholic Beverages)', 'Restaurants and Other Eating Places', 'Specialty Food Services']
filtered_df = merged_df[merged_df['top_category'].isin(topcategory_list)]
filtered_df=filtered_df.reset_index().drop('index', axis =1)
filtered_df.head()

,placekey,raw_visit_counts,raw_visitor_counts,visits_by_day,visits_by_each_hour,poi_cbg,visitor_home_cbgs,visitor_daytime_cbgs,distance_from_home,normalized_visits_by_state_scaling,date_range_start,date_range_end,median_dwell,bucketed_dwell_times,latitude,longitude,top_category,open_hours,location_name
0,zzy-222@62j-sjv-85f,141.0,121.0,"[14, 9, 10, 18, 18, 43, 29]","[4,4,2,2,1,2,0,0,10,11,14,13,20,17,21,22,20,16...",250173537004,"{""250173537003"":11,""250173537005"":6,""250173538...","{""250173537003"":8,""250173537005"":7,""2502141780...",6643.0,3145.31624540,2021-10-01T00:00:00-04:00,2021-11-01T00:00:00-04:00,12.0,"{""<5"":3,""5-10"":56,""11-20"":38,""21-60"":28,""61-12...",42.373312,-71.119151,Restaurants and Other Eating Places,"{ ""Mon"": [[""5:00"", ""23:00""]], ""Tue"": [[""5:00"",...",Starbucks
1,zzw-224@62j-sj2-s5z,22.0,18.0,"[2, 1, 4, 3, 4, 6, 2]","[5,3,3,3,2,2,2,2,5,4,4,4,4,4,4,5,6,5,2,4,7,10,...",250173531011,"{""250010121011"":4,""250250104051"":4}","{""250010121011"":4,""340230051002"":4,""4400302090...",23581.0,490.75856311,2021-10-01T00:00:00-04:00,2021-11-01T00:00:00-04:00,153.0,"{""<5"":0,""5-10"":3,""11-20"":2,""21-60"":2,""61-120"":...",42.363572,-71.101336,Restaurants and Other Eating Places,"{ ""Mon"": [[""11:30"", ""22:00""]], ""Tue"": [], ""Wed...",Mary Chung Restaurant
2,245-222@62j-sj3-9zz,19.0,19.0,"[2, 2, 1, 0, 5, 6, 3]","[0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,4,6,3,3,4,4,3,2]",250173513003,"{""250251011023"":4,""250173371023"":4,""2501735210...","{""250173621001"":4,""250214008003"":4,""2501731730...",7662.0,423.83694087,2021-10-01T00:00:00-04:00,2021-11-01T00:00:00-04:00,44.0,"{""<5"":1,""5-10"":3,""11-20"":2,""21-60"":7,""61-120"":...",42.379954,-71.094380,Drinking Places (Alcoholic Beverages),"{ ""Mon"": [], ""Tue"": [[""16:00"", ""24:00""]], ""Wed...",Backbar
3,224-222@62j-sjv-8qf,374.0,171.0,"[43, 55, 63, 55, 63, 57, 38]","[25,16,8,5,3,5,17,32,37,38,36,48,55,38,50,40,3...",250173539003,"{""250173539003"":29,""250173539001"":22,""25017353...","{""250173539003"":24,""250173539001"":21,""25017353...",1327.0,8342.89557291,2021-10-01T00:00:00-04:00,2021-11-01T00:00:00-04:00,41.0,"{""<5"":7,""5-10"":46,""11-20"":64,""21-60"":113,""61-1...",42.371488,-71.116847,Restaurants and Other Eating Places,"{ ""Mon"": [[""8:00"", ""16:00""]], ""Tue"": [[""8:00"",...",Blue Bottle Coffee
4,223-222@62j-sgg-kfz,187.0,93.0,"[21, 21, 31, 28, 37, 20, 29]","[23,25,24,26,26,29,61,70,67,67,67,56,60,59,51,...",250173534002,"{""250173534002"":6,""250251011022"":5,""2501735470...","{""250173534002"":5,""250235114003"":4,""2502512050...",9869.0,4171.44778645,2021-10-01T00:00:00-04:00,2021-11-01T00:00:00-04:00,66.0,"{""<5"":8,""5-10"":45,""11-20"":19,""21-60"":18,""61-12...",42.361149,-71.115516,Restaurants and Other Eating Places,NaN,Baraka Cafe


In [148]:
# From filtered file -> Change visits by hour (1 column) to actual columns (24: one by hour)
hour_df = pd.DataFrame()
hour_df ['placekey'] = filtered_df ['placekey'].copy()
hour_df ['poi_cbg'] = filtered_df['poi_cbg'].copy()
for i in range(len(filtered_df)):
    aux = json.loads(filtered_df['visits_by_each_hour'][i])
    for j in range(24):
        hour_df.loc[i, j] = aux[j]
hour_df.head()

,placekey,poi_cbg,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,zzy-222@62j-sjv-85f,250173537004,4.0,4.0,2.0,2.0,1.0,2.0,0.0,0.0,...,21.0,22.0,20.0,16.0,17.0,8.0,10.0,6.0,4.0,3.0
1,zzw-224@62j-sj2-s5z,250173531011,5.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,...,4.0,5.0,6.0,5.0,2.0,4.0,7.0,10.0,8.0,8.0
2,245-222@62j-sj3-9zz,250173513003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,4.0,6.0,3.0,3.0,4.0,4.0,3.0,2.0
3,224-222@62j-sjv-8qf,250173539003,25.0,16.0,8.0,5.0,3.0,5.0,17.0,32.0,...,50.0,40.0,34.0,42.0,61.0,52.0,38.0,54.0,55.0,40.0
4,223-222@62j-sgg-kfz,250173534002,23.0,25.0,24.0,26.0,26.0,29.0,61.0,70.0,...,51.0,32.0,21.0,22.0,18.0,10.0,12.0,17.0,20.0,22.0


In [149]:
# Change the type of numeric values and sort data by blockgroup
for i in range(24):
    hour_df[i] = hour_df[i].astype('Int64')

hour_df = hour_df.sort_values(by='poi_cbg', ascending = True).reset_index().drop('index', axis =1)
hour_df.head()

,placekey,poi_cbg,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,22g-222@62j-sk2-pjv,250173512031,8,9,8,8,8,12,12,8,...,6,4,3,7,7,7,8,7,8,7
1,237-222@62j-sk2-pd9,250173512031,41,39,38,39,37,36,37,35,...,34,35,37,38,41,44,37,42,45,44
2,24h-222@62j-sj3-9j9,250173512033,9,9,9,9,9,10,11,10,...,9,6,3,8,11,15,14,14,14,14
3,22m-222@62j-sj3-9j9,250173512033,1,1,1,1,1,1,1,2,...,2,5,8,7,8,3,8,13,9,4
4,235-222@62j-sj3-b6k,250173512042,0,1,0,0,0,0,0,10,...,6,5,7,3,9,7,1,0,0,0


In [150]:
# Group by blockgroup and sum the visits by blockgroup
hour_bg_groupped = hour_df.groupby('poi_cbg').sum()
hour_bg_groupped = hour_bg_groupped.reset_index()
hour_bg_groupped.head()

,poi_cbg,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,250173512031,49,48,46,47,45,48,49,43,44,...,40,39,40,45,48,51,45,49,53,51
1,250173512033,10,10,10,10,10,11,12,12,12,...,11,11,11,15,19,18,22,27,23,18
2,250173512042,1,2,1,1,1,1,1,12,3,...,10,11,11,7,13,10,3,2,1,1
3,250173512043,3,4,3,2,2,2,3,2,2,...,14,15,16,18,16,7,6,6,3,3
4,250173513003,37,19,13,14,14,16,21,20,26,...,41,41,76,84,93,102,101,106,84,52


In [151]:
# Calculate Probabilities
list_bg = hour_bg_groupped['poi_cbg'].unique().tolist()
aux_poi_cbg = pd.DataFrame()
for i in range(len(hour_df)):
    for j in range(24):
        bg = hour_bg_groupped[hour_bg_groupped['poi_cbg'] == hour_df.at[i, 'poi_cbg']]
        divider = bg[j].item()
        if divider != 0:
            hour_df.at[i, 'Prob_'+str(j)] = round(hour_df.loc[i,j]/divider,3)
        else:
            hour_df.at[i, 'Prob_'+str(j)] = 0
hour_df.head()

,placekey,poi_cbg,0,1,2,3,4,5,6,7,...,Prob_14,Prob_15,Prob_16,Prob_17,Prob_18,Prob_19,Prob_20,Prob_21,Prob_22,Prob_23
0,22g-222@62j-sk2-pjv,250173512031,8,9,8,8,8,12,12,8,...,0.150,0.103,0.075,0.156,0.146,0.137,0.178,0.143,0.151,0.137
1,237-222@62j-sk2-pd9,250173512031,41,39,38,39,37,36,37,35,...,0.850,0.897,0.925,0.844,0.854,0.863,0.822,0.857,0.849,0.863
2,24h-222@62j-sj3-9j9,250173512033,9,9,9,9,9,10,11,10,...,0.818,0.545,0.273,0.533,0.579,0.833,0.636,0.519,0.609,0.778
3,22m-222@62j-sj3-9j9,250173512033,1,1,1,1,1,1,1,2,...,0.182,0.455,0.727,0.467,0.421,0.167,0.364,0.481,0.391,0.222
4,235-222@62j-sj3-b6k,250173512042,0,1,0,0,0,0,0,10,...,0.600,0.455,0.636,0.429,0.692,0.700,0.333,0.000,0.000,0.000


In [152]:
# Merge Probabilities with places to have lat, lon
merge_hour_df=hour_df.merge(places_df[['placekey','latitude','longitude', 'top_category']], how='left', left_on='placekey', right_on='placekey')
merge_hour_df.head()

,placekey,poi_cbg,0,1,2,3,4,5,6,7,...,Prob_17,Prob_18,Prob_19,Prob_20,Prob_21,Prob_22,Prob_23,latitude,longitude,top_category
0,22g-222@62j-sk2-pjv,250173512031,8,9,8,8,8,12,12,8,...,0.156,0.146,0.137,0.178,0.143,0.151,0.137,42.378182,-71.106010,Restaurants and Other Eating Places
1,237-222@62j-sk2-pd9,250173512031,41,39,38,39,37,36,37,35,...,0.844,0.854,0.863,0.822,0.857,0.849,0.863,42.377298,-71.104867,Restaurants and Other Eating Places
2,24h-222@62j-sj3-9j9,250173512033,9,9,9,9,9,10,11,10,...,0.533,0.579,0.833,0.636,0.519,0.609,0.778,42.374521,-71.101502,Drinking Places (Alcoholic Beverages)
3,22m-222@62j-sj3-9j9,250173512033,1,1,1,1,1,1,1,2,...,0.467,0.421,0.167,0.364,0.481,0.391,0.222,42.374469,-71.101504,Restaurants and Other Eating Places
4,235-222@62j-sj3-b6k,250173512042,0,1,0,0,0,0,0,10,...,0.429,0.692,0.700,0.333,0.000,0.000,0.000,42.379918,-71.096778,Restaurants and Other Eating Places


In [153]:
# Drop visits per hour and keep probabilities
drop_list=[ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,  22, 23]
merge_hour_df=merge_hour_df.drop(drop_list,axis=1)
merge_hour_df.head()

,placekey,poi_cbg,Prob_0,Prob_1,Prob_2,Prob_3,Prob_4,Prob_5,Prob_6,Prob_7,...,Prob_17,Prob_18,Prob_19,Prob_20,Prob_21,Prob_22,Prob_23,latitude,longitude,top_category
0,22g-222@62j-sk2-pjv,250173512031,0.163,0.188,0.174,0.17,0.178,0.250,0.245,0.186,...,0.156,0.146,0.137,0.178,0.143,0.151,0.137,42.378182,-71.106010,Restaurants and Other Eating Places
1,237-222@62j-sk2-pd9,250173512031,0.837,0.812,0.826,0.83,0.822,0.750,0.755,0.814,...,0.844,0.854,0.863,0.822,0.857,0.849,0.863,42.377298,-71.104867,Restaurants and Other Eating Places
2,24h-222@62j-sj3-9j9,250173512033,0.900,0.900,0.900,0.90,0.900,0.909,0.917,0.833,...,0.533,0.579,0.833,0.636,0.519,0.609,0.778,42.374521,-71.101502,Drinking Places (Alcoholic Beverages)
3,22m-222@62j-sj3-9j9,250173512033,0.100,0.100,0.100,0.10,0.100,0.091,0.083,0.167,...,0.467,0.421,0.167,0.364,0.481,0.391,0.222,42.374469,-71.101504,Restaurants and Other Eating Places
4,235-222@62j-sj3-b6k,250173512042,0.000,0.500,0.000,0.00,0.000,0.000,0.000,0.833,...,0.429,0.692,0.700,0.333,0.000,0.000,0.000,42.379918,-71.096778,Restaurants and Other Eating Places


# REPLICA DATA ANALYSIS

In [154]:
# INSERT REPLICA TRIPS DATA
filepath_Replica_Trips = ".././02 Data/Replica/Trips/BigQuery/Tangible_Table/"
replica_trips_df = pd.read_csv(filepath_Replica_Trips + 'replica_tangible_table_bq_df.csv')
replica_trips_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,start_local_hour,end_time,end_local_hour,duration_minutes,distance_miles,vehicle_type,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2
0,5.65274152e+18,HOME,MAINTENANCE,2023-02-03 13:13:18,13,2023-02-03 13:15:13,13,1,0.55918461,NaN,...,42.3721570,-71.11732789,mixed_use,retail,commercial,office,mixed_use,multi_family,residential,multi_family
1,1.30452013e+19,SCHOOL,SHOP,2023-02-03 14:11:05,14,2023-02-03 14:14:25,14,3,0.91876304,NaN,...,42.3753750,-71.11070189,mixed_use,retail,commercial,retail,civic_institutional,education,civic_institutional,education
2,7.51852191e+18,HOME,EAT,2023-02-03 19:13:27,19,2023-02-03 19:34:44,19,21,1.10263608,NaN,...,42.3585020,-71.08549670,commercial,retail,commercial,retail,civic_institutional,education,civic_institutional,education
3,1.24914904e+19,HOME,SOCIAL,2023-02-03 10:17:02,10,2023-02-03 10:20:50,10,3,1.40175431,NaN,...,42.3653525,-71.08969719,civic_institutional,education,commercial,retail,residential,multi_family,residential,multi_family
4,1.61620345e+19,SHOP,WORK,2023-02-03 18:26:56,18,2023-02-03 18:28:18,18,1,0.07098729,NaN,...,42.3697630,-71.08985094,mixed_use,non_retail_attraction,civic_institutional,healthcare,mixed_use,office,commercial,office


In [155]:
drop_list = ['start_local_hour', 'end_local_hour','origin_building_use_l1', 'origin_building_use_l2', 'destination_building_use_l1','destination_building_use_l2']
replica_trips_df=replica_trips_df.drop(drop_list, axis=1)
replica_trips_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2
0,5.65274152e+18,HOME,MAINTENANCE,2023-02-03 13:13:18,2023-02-03 13:15:13,1,0.55918461,NaN,250173538004,42.3683430,-71.10977366,250173537004,42.3721570,-71.11732789,mixed_use,retail,mixed_use,multi_family
1,1.30452013e+19,SCHOOL,SHOP,2023-02-03 14:11:05,2023-02-03 14:14:25,3,0.91876304,NaN,250173537005,42.3739040,-71.11366683,250173537002,42.3753750,-71.11070189,mixed_use,retail,civic_institutional,education
2,7.51852191e+18,HOME,EAT,2023-02-03 19:13:27,2023-02-03 19:34:44,21,1.10263608,NaN,250173530001,42.3662725,-71.10594473,250173531022,42.3585020,-71.08549670,commercial,retail,civic_institutional,education
3,1.24914904e+19,HOME,SOCIAL,2023-02-03 10:17:02,2023-02-03 10:20:50,3,1.40175431,NaN,250173537005,42.3739040,-71.11366683,250173524001,42.3653525,-71.08969719,civic_institutional,education,residential,multi_family
4,1.61620345e+19,SHOP,WORK,2023-02-03 18:26:56,2023-02-03 18:28:18,1,0.07098729,NaN,250173524001,42.3653525,-71.08969719,250173526001,42.3697630,-71.08985094,mixed_use,non_retail_attraction,mixed_use,office


In [156]:
# Filter trips by trip purpose = EAT
replica_eat_df = replica_trips_df[replica_trips_df['travel_purpose'] == 'EAT']
replica_eat_df=replica_eat_df.reset_index().drop('index',axis=1)
replica_eat_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2
0,4.37230743e+18,EAT,HOME,2023-02-03 19:19:07,2023-02-03 19:20:25,1,0.35610543,NaN,250173538004,42.3683430,-71.10977366,250173530001,42.3662725,-71.10594473,mixed_use,multi_family,commercial,retail
1,1.31959928e+19,EAT,SCHOOL,2023-02-03 13:56:16,2023-02-03 14:02:49,6,1.65805085,NaN,250173537005,42.3739040,-71.11366683,250173527002,42.3736705,-71.09474173,residential,multi_family,mixed_use,industrial
2,5.35644598e+17,EAT,SOCIAL,2023-02-03 14:48:57,2023-02-03 14:51:01,2,0.49632586,NaN,250173525001,42.3691315,-71.09751020,250173527001,42.3731030,-71.08964300,residential,single_family,mixed_use,retail
3,6.87737414e+18,EAT,WORK_FROM_HOME,2023-02-03 14:38:43,2023-02-03 14:44:43,6,1.51005010,NaN,250173538001,42.3710035,-71.10737614,250173531011,42.3616040,-71.09919947,residential,multi_family,commercial,retail
4,1.79669229e+19,EAT,HOME,2023-02-03 04:58:01,2023-02-03 04:59:48,1,0.70711457,NaN,250173528001,42.3705775,-71.09945649,250173527002,42.3736705,-71.09474173,residential,single_family,commercial,retail


In [157]:
len(replica_eat_df)

7987

In [158]:
# # Destination landuse commercial
# replica_eat_df = replica_eat_df[replica_eat_df['destination_land_use_l1']=='commercial']
# replica_eat_df=replica_eat_df.reset_index().drop('index',axis=1)
# replica_eat_df.head()

In [159]:
# Change blockgroup data to int so that it matches blockgroup data in Safegraph files
replica_eat_df['destination_bgrp']=replica_eat_df['destination_bgrp'].astype(np.int64)
# Generate a new column with the start hour of the trips to make a match
replica_eat_df['start_hour'] = replica_eat_df['start_time'].str[:2].astype(np.int64)
replica_eat_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour
0,4.37230743e+18,EAT,HOME,2023-02-03 19:19:07,2023-02-03 19:20:25,1,0.35610543,NaN,250173538004,42.3683430,-71.10977366,250173530001,42.3662725,-71.10594473,mixed_use,multi_family,commercial,retail,20
1,1.31959928e+19,EAT,SCHOOL,2023-02-03 13:56:16,2023-02-03 14:02:49,6,1.65805085,NaN,250173537005,42.3739040,-71.11366683,250173527002,42.3736705,-71.09474173,residential,multi_family,mixed_use,industrial,20
2,5.35644598e+17,EAT,SOCIAL,2023-02-03 14:48:57,2023-02-03 14:51:01,2,0.49632586,NaN,250173525001,42.3691315,-71.09751020,250173527001,42.3731030,-71.08964300,residential,single_family,mixed_use,retail,20
3,6.87737414e+18,EAT,WORK_FROM_HOME,2023-02-03 14:38:43,2023-02-03 14:44:43,6,1.51005010,NaN,250173538001,42.3710035,-71.10737614,250173531011,42.3616040,-71.09919947,residential,multi_family,commercial,retail,20
4,1.79669229e+19,EAT,HOME,2023-02-03 04:58:01,2023-02-03 04:59:48,1,0.70711457,NaN,250173528001,42.3705775,-71.09945649,250173527002,42.3736705,-71.09474173,residential,single_family,commercial,retail,20


In [160]:
# Assign a restaurant from blockgroup based on probabilities
from random import choices
len_trips = len(replica_eat_df)
trips_bg = pd.DataFrame(replica_eat_df['destination_bgrp'].values)
trips_bg = trips_bg.rename(columns={0: "destination_bgrp"}, errors="raise")
probabs = pd.DataFrame()
cho = np.empty(len_trips, dtype=object)
count=0
for i in range (len_trips):
    trip_bg = trips_bg['destination_bgrp'].values[i]
    restaurants = merge_hour_df[merge_hour_df['poi_cbg'] == trip_bg]
    
    probabs = restaurants[['placekey', 'Prob_' +str(replica_eat_df['start_hour'].values[i])]]
    aux=0
    for j in range(len(probabs)):
        if probabs['Prob_' +str(replica_eat_df['start_hour'].values[i])].values[j]!=0.0 :
            aux = 1
            break
        else:
            aux=0
    if aux==1:
        choice=choices(probabs['placekey'].to_list(), probabs ['Prob_' + str(replica_eat_df['start_hour'].values[i])].to_list())
        choice_aux = choice[0]
        cho[i]=choice_aux
    else:
        cho[i]='nothing'
        count=count+1

replica_eat_df['placekey']=cho
replica_eat_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey
0,4.37230743e+18,EAT,HOME,2023-02-03 19:19:07,2023-02-03 19:20:25,1,0.35610543,NaN,250173538004,42.3683430,-71.10977366,250173530001,42.3662725,-71.10594473,mixed_use,multi_family,commercial,retail,20,22y-222@62j-sj2-t35
1,1.31959928e+19,EAT,SCHOOL,2023-02-03 13:56:16,2023-02-03 14:02:49,6,1.65805085,NaN,250173537005,42.3739040,-71.11366683,250173527002,42.3736705,-71.09474173,residential,multi_family,mixed_use,industrial,20,24s-222@62j-sj3-85f
2,5.35644598e+17,EAT,SOCIAL,2023-02-03 14:48:57,2023-02-03 14:51:01,2,0.49632586,NaN,250173525001,42.3691315,-71.09751020,250173527001,42.3731030,-71.08964300,residential,single_family,mixed_use,retail,20,229-222@62j-sj2-6zf
3,6.87737414e+18,EAT,WORK_FROM_HOME,2023-02-03 14:38:43,2023-02-03 14:44:43,6,1.51005010,NaN,250173538001,42.3710035,-71.10737614,250173531011,42.3616040,-71.09919947,residential,multi_family,commercial,retail,20,232-222@62j-sj2-c3q
4,1.79669229e+19,EAT,HOME,2023-02-03 04:58:01,2023-02-03 04:59:48,1,0.70711457,NaN,250173528001,42.3705775,-71.09945649,250173527002,42.3736705,-71.09474173,residential,single_family,commercial,retail,20,23b-223@62j-sj3-85f


# MERGE SAFEGRAPH AND REPLICA

In [161]:
# Add latitude and longitude of every assigned restaurant
eat_df = replica_eat_df.merge(filtered_df[['placekey','latitude', 'longitude']], how='left', left_on='placekey', right_on='placekey')
eat_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,latitude,longitude
0,4.37230743e+18,EAT,HOME,2023-02-03 19:19:07,2023-02-03 19:20:25,1,0.35610543,NaN,250173538004,42.3683430,...,42.3662725,-71.10594473,mixed_use,multi_family,commercial,retail,20,22y-222@62j-sj2-t35,42.365915,-71.105008
1,1.31959928e+19,EAT,SCHOOL,2023-02-03 13:56:16,2023-02-03 14:02:49,6,1.65805085,NaN,250173537005,42.3739040,...,42.3736705,-71.09474173,residential,multi_family,mixed_use,industrial,20,24s-222@62j-sj3-85f,42.373189,-71.095430
2,5.35644598e+17,EAT,SOCIAL,2023-02-03 14:48:57,2023-02-03 14:51:01,2,0.49632586,NaN,250173525001,42.3691315,...,42.3731030,-71.08964300,residential,single_family,mixed_use,retail,20,229-222@62j-sj2-6zf,42.372721,-71.091448
3,6.87737414e+18,EAT,WORK_FROM_HOME,2023-02-03 14:38:43,2023-02-03 14:44:43,6,1.51005010,NaN,250173538001,42.3710035,...,42.3616040,-71.09919947,residential,multi_family,commercial,retail,20,232-222@62j-sj2-c3q,42.361999,-71.098471
4,1.79669229e+19,EAT,HOME,2023-02-03 04:58:01,2023-02-03 04:59:48,1,0.70711457,NaN,250173528001,42.3705775,...,42.3736705,-71.09474173,residential,single_family,commercial,retail,20,23b-223@62j-sj3-85f,42.373154,-71.095093


# REASSIGNMENT

In [162]:
# Trips where the restaurants from the blockgroup have probability=0 to happen
nonassigned_df = eat_df[eat_df['placekey']=='nothing']
drop_list = ['placekey','latitude','longitude']
nonassigned_df = nonassigned_df.drop(drop_list, axis=1)
nonassigned_df = nonassigned_df.reset_index().drop('index',axis=1)
nonassigned_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour
0,1.60389730e+19,EAT,WORK_FROM_HOME,2023-02-03 11:54:01,2023-02-03 12:00:28,6,1.42016212,NaN,250173512033,42.375701,-71.09907857,250173532001,42.3602035,-71.10486411,residential,multi_family,civic_institutional,education,20
1,5.32233230e+18,EAT,WORK_FROM_HOME,2023-02-03 10:00:41,2023-02-03 10:21:22,20,1.07120922,NaN,250173541002,42.373252,-71.12708263,250173538002,42.3694630,-71.10833580,residential,multi_family,commercial,office,20
2,9.37128197e+18,EAT,HOME,2023-02-03 17:41:04,2023-02-03 17:43:57,2,0.67682589,NaN,250173536001,42.379243,-71.11028047,250173512034,42.3759925,-71.10314558,residential,multi_family,commercial,retail,20
3,1.18150477e+19,EAT,SCHOOL,2023-02-03 18:10:28,2023-02-03 18:22:32,12,0.96349744,NaN,250173527001,42.373103,-71.08964300,250173512034,42.3759925,-71.10314558,mixed_use,office,commercial,retail,20
4,1.42311917e+18,EAT,HOME,2023-02-03 14:08:36,2023-02-03 14:09:29,0,0.04603912,NaN,250173538004,42.368343,-71.10977366,250173538002,42.3694630,-71.10833580,mixed_use,multi_family,commercial,office,20


In [163]:
len(nonassigned_df)

243

In [164]:
# Get the visits by day to the restaurants in the blockgroups with trips not assigned
blockgroups = nonassigned_df['destination_bgrp'].unique()
week_df = pd.DataFrame()
fweek_df = pd.DataFrame()
week_df ['placekey'] = filtered_df ['placekey'].copy()
week_df ['poi_cbg'] = filtered_df['poi_cbg'].copy()
aux_df = pd.DataFrame()
for i in range (len(blockgroups)):
    aux_df = week_df[week_df['poi_cbg'] == blockgroups[i]]
    fweek_df = pd.concat([fweek_df,aux_df])
fweek_df = fweek_df.merge(filtered_df[['placekey', 'visits_by_day']], how='left', left_on='placekey', right_on='placekey')
fweek_df

,placekey,poi_cbg,visits_by_day
0,265-222@62j-sgg-jvz,250173532002,"[3, 0, 1, 5, 3, 1, 2]"
1,23c-222@62j-sgg-k75,250173532002,"[2, 2, 5, 3, 2, 0, 1]"


In [165]:
# Calculate the total visits per week for those restaurants
for i in range(len(fweek_df)):
    aux = json.loads(fweek_df['visits_by_day'][i])
    for j in range(7):
        fweek_df.loc[i, j] = aux[j]
for i in range(7):
    fweek_df[i] = fweek_df[i].astype('Int64')
fweek_df=fweek_df.rename(columns={0: "Monday"}, errors="raise")        
fweek_df=fweek_df.rename(columns={1: "Tuesday"}, errors="raise") 
fweek_df=fweek_df.rename(columns={2: "Wednesday"}, errors="raise") 
fweek_df=fweek_df.rename(columns={3: "Thrusday"}, errors="raise") 
fweek_df=fweek_df.rename(columns={4: "Friday"}, errors="raise") 
fweek_df=fweek_df.rename(columns={5: "Saturday"}, errors="raise") 
fweek_df=fweek_df.rename(columns={6: "Sunday"}, errors="raise") 
fweek_df['Total_Visits']=fweek_df['Monday']+fweek_df['Tuesday']+fweek_df['Wednesday']+fweek_df['Thrusday']+fweek_df['Friday']+fweek_df['Saturday']+fweek_df['Sunday']
fweek_df = fweek_df.sort_values(by='poi_cbg', ascending = True).reset_index().drop('index', axis =1)
fweek_df.head()

,placekey,poi_cbg,visits_by_day,Monday,Tuesday,Wednesday,Thrusday,Friday,Saturday,Sunday,Total_Visits
0,265-222@62j-sgg-jvz,250173532002,"[3, 0, 1, 5, 3, 1, 2]",3,0,1,5,3,1,2,15
1,23c-222@62j-sgg-k75,250173532002,"[2, 2, 5, 3, 2, 0, 1]",2,2,5,3,2,0,1,15


In [166]:
# Drop visits per hour and keep probabilities
drop_list=['visits_by_day', 'Monday', 'Tuesday', 'Wednesday', 'Thrusday', 'Friday', 'Saturday', 'Sunday']
fweek_df=fweek_df.drop(drop_list,axis=1)
fweek_df=fweek_df.merge(filtered_df[['placekey', 'open_hours']], how='left', left_on='placekey', right_on='placekey')
fweek_df=fweek_df.dropna().reset_index().drop('index',axis=1)
fweek_df.head()

,placekey,poi_cbg,Total_Visits,open_hours


In [167]:
# Security copy
fweek_df_copy = fweek_df.copy()
fweek_df_copy=fweek_df_copy.dropna().reset_index().drop('index',axis=1)
fweek_df_copy.head()

,placekey,poi_cbg,Total_Visits,open_hours


In [168]:
# Extracted open and close hours of each restaurant
import json
a = pd.DataFrame()
x = pd.DataFrame()
fweek_df_copy['Open']=' '
fweek_df_copy['Close']=' '
if fweek_df_copy.empty == False:
    for i in range (len(fweek_df_copy)):
        a = json.loads(fweek_df_copy['open_hours'][i])
        x = a.get('Tue')
        if len(x)==0:
            x = a.get('Mon')
            if len(x)==0:
                x = a.get('Wed')
                if len(x)==0:
                    x = a.get('Thu')
                    if len(x)==0:
                        x = a.get('Fri')
                    else:
                        fweek_df_copy.at[i,'Open']=x[0][0]
                        fweek_df_copy.at[i,'Close']=x[0][1]
                else:
                    fweek_df_copy.at[i,'Open']=x[0][0]
                    fweek_df_copy.at[i,'Close']=x[0][1]
            else:
                fweek_df_copy.at[i,'Open']=x[0][0]
                fweek_df_copy.at[i,'Close']=x[0][1]
        else:
            if x[0][1]=='24:00':
                x[0][1]='0:00'
                fweek_df_copy.at[i,'Close']=x[0][1]
            else:
                fweek_df_copy.at[i,'Close']=x[0][1]
            if x[0][0]=='0:00':
                x = a.get('Mon')
                if x[0][0]=='0:00':
                    size=len(x)
                    if size==1:
                        fweek_df_copy.at[i,'Open']=x[0][0]
                    elif size==2:
                        fweek_df_copy.at[i,'Open']=x[1][0]
                else:
                    fweek_df_copy.at[i,'Open']=x[0][0]
            else:
                fweek_df_copy.at[i,'Open']=x[0][0]
drop_list=['open_hours']
fweek_df_copy=fweek_df_copy.drop(drop_list, axis=1)
fweek_df_copy.head()

,placekey,poi_cbg,Total_Visits,Open,Close


In [169]:
# Replace : by . to change type of variable
if fweek_df_copy.empty == False:
    fweek_df_copy['Open'] = fweek_df_copy['Open'].replace({':' : '.'}, regex=True)
    fweek_df_copy['Close'] = fweek_df_copy['Close'].replace({':' : '.'}, regex=True)
    fweek_df_copy.head()

In [170]:
# Change open and close variables for a numeric one instead of a string
if fweek_df_copy.empty == False:
    fweek_df_copy['Open'] = pd.to_numeric(fweek_df_copy['Open'],errors='coerce')
    fweek_df_copy['Close'] = pd.to_numeric(fweek_df_copy['Close'],errors='coerce')
    fweek_df_copy.head()

In [171]:
# Calculate visits per hour based on open hours and total visits per week
week_hours_df=fweek_df_copy.copy()
length=len(week_hours_df)
if fweek_df_copy.empty == False:
    for i in range(24):
        week_hours_df[i]=' '
    for i in range(length):
        hours_open = 0
        for j in range (24):
            if week_hours_df.at[i,'Close']<week_hours_df.at[i,'Open']:
                hours_open = 24 - abs (week_hours_df.at[i,'Close']-week_hours_df.at[i,'Open'])
                if j>=week_hours_df.at[i,'Open']:
                        week_hours_df.at[i,j]=(week_hours_df.at[i,'Total_Visits']/hours_open)
                else:
                    if j>=week_hours_df.at[i,'Close']:
                        week_hours_df.at[i,j]=0
                    else:
                        week_hours_df.at[i,j]=0
            else:
                if j>=week_hours_df.at[i,'Open']:
                    hours_open = week_hours_df.at[i,'Close']-week_hours_df.at[i,'Open']
                    if j<fweek_df_copy['Close'][i]:
                        week_hours_df.at[i,j]=(week_hours_df.at[i,'Total_Visits']/hours_open)
                    else:
                        week_hours_df.at[i,j]=0
                else:
                    week_hours_df.at[i,j]=0
        if week_hours_df.at[i,'Close']<week_hours_df.at[i,'Open']:
                aux=week_hours_df.at[i,'Close'].astype(int)
                for k in range(aux):
                    week_hours_df.at[i,k]=(week_hours_df.at[i,'Total_Visits']/hours_open)
    week_hours_df.head()

In [172]:
# Copy
if fweek_df_copy.empty == False:
    wh_df=week_hours_df.copy()
    wh_df.head()

In [173]:
# Change visits per hour from object to float
if fweek_df_copy.empty == False:
    for i in range(24):
        wh_df[i]=wh_df[i].astype('Float64')

In [174]:
# Group by blockgroup and get the total weekly visits per week
if fweek_df_copy.empty == False:
    week_bg_groupped = wh_df.groupby(wh_df['poi_cbg']).sum()
    week_bg_groupped = week_bg_groupped.reset_index()
    week_bg_groupped.head()

In [175]:
# Calculate Probabilities based on week visits
if fweek_df_copy.empty == False:
    for i in range(len(wh_df)):
        for j in range(24):
            bg_week = week_bg_groupped[week_bg_groupped['poi_cbg'] == wh_df.at[i, 'poi_cbg']]
            divider = bg_week[j].item()
            if divider != 0:
                wh_df.at[i, 'Prob_'+str(j)] = round(wh_df.loc[i,j]/divider,3)
            else:
                wh_df.at[i, 'Prob_'+str(j)] = 0
    wh_df.head()

In [176]:
# Assign a restaurant from blockgroup based on probabilities
from random import choices
if fweek_df_copy.empty == False:
    len_trips_na = len(nonassigned_df)
    trips_bg_na = pd.DataFrame(nonassigned_df['destination_bgrp'].values)
    trips_bg_na = trips_bg_na.rename(columns={0: "destination_bgrp"}, errors="raise")
    probabs_na = pd.DataFrame()
    cho_na = np.empty(len_trips_na, dtype=object)
    count=0
    for i in range (len_trips_na):
        trip_bg_na = trips_bg_na['destination_bgrp'].values[i]
        restaurants_na = wh_df[wh_df['poi_cbg'] == trip_bg_na]
        probabs_na = restaurants_na[['placekey', 'Prob_' +str(nonassigned_df['start_hour'].values[i])]]
        aux=0
        for j in range(len(probabs_na)):
            if probabs_na['Prob_' +str(nonassigned_df['start_hour'].values[i])].values[j]!=0.0 :
                aux = 1
                break
            else:
                aux=0
        if aux==1:
            choice_na=choices(probabs_na['placekey'].to_list(), probabs_na['Prob_' + str(nonassigned_df['start_hour'].values[i])].to_list())
            choice_aux = choice_na[0]
            cho_na[i]=choice_aux
        else:
            cho_na[i]='nothing'
            count=count+1

    nonassigned_df['placekey']=cho_na
else:
    nonassigned_df['placekey'] = 'nothing'
    
nonassigned_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey
0,1.60389730e+19,EAT,WORK_FROM_HOME,2023-02-03 11:54:01,2023-02-03 12:00:28,6,1.42016212,NaN,250173512033,42.375701,-71.09907857,250173532001,42.3602035,-71.10486411,residential,multi_family,civic_institutional,education,20,nothing
1,5.32233230e+18,EAT,WORK_FROM_HOME,2023-02-03 10:00:41,2023-02-03 10:21:22,20,1.07120922,NaN,250173541002,42.373252,-71.12708263,250173538002,42.3694630,-71.10833580,residential,multi_family,commercial,office,20,nothing
2,9.37128197e+18,EAT,HOME,2023-02-03 17:41:04,2023-02-03 17:43:57,2,0.67682589,NaN,250173536001,42.379243,-71.11028047,250173512034,42.3759925,-71.10314558,residential,multi_family,commercial,retail,20,nothing
3,1.18150477e+19,EAT,SCHOOL,2023-02-03 18:10:28,2023-02-03 18:22:32,12,0.96349744,NaN,250173527001,42.373103,-71.08964300,250173512034,42.3759925,-71.10314558,mixed_use,office,commercial,retail,20,nothing
4,1.42311917e+18,EAT,HOME,2023-02-03 14:08:36,2023-02-03 14:09:29,0,0.04603912,NaN,250173538004,42.368343,-71.10977366,250173538002,42.3694630,-71.10833580,mixed_use,multi_family,commercial,office,20,nothing


In [177]:
l_non_assigned = len(nonassigned_df)
l_nothing = len(nonassigned_df[nonassigned_df['placekey']=='nothing'])

In [178]:
l_nothing

243

In [179]:
dropped_trips = nonassigned_df[nonassigned_df['placekey']=='nothing'].reset_index().drop('index',axis=1)
dropped_trips

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey
0,1.60389730e+19,EAT,WORK_FROM_HOME,2023-02-03 11:54:01,2023-02-03 12:00:28,6,1.42016212,NaN,250173512033,42.375701,-71.09907857,250173532001,42.3602035,-71.10486411,residential,multi_family,civic_institutional,education,20,nothing
1,5.32233230e+18,EAT,WORK_FROM_HOME,2023-02-03 10:00:41,2023-02-03 10:21:22,20,1.07120922,NaN,250173541002,42.373252,-71.12708263,250173538002,42.3694630,-71.10833580,residential,multi_family,commercial,office,20,nothing
2,9.37128197e+18,EAT,HOME,2023-02-03 17:41:04,2023-02-03 17:43:57,2,0.67682589,NaN,250173536001,42.379243,-71.11028047,250173512034,42.3759925,-71.10314558,residential,multi_family,commercial,retail,20,nothing
3,1.18150477e+19,EAT,SCHOOL,2023-02-03 18:10:28,2023-02-03 18:22:32,12,0.96349744,NaN,250173527001,42.373103,-71.08964300,250173512034,42.3759925,-71.10314558,mixed_use,office,commercial,retail,20,nothing
4,1.42311917e+18,EAT,HOME,2023-02-03 14:08:36,2023-02-03 14:09:29,0,0.04603912,NaN,250173538004,42.368343,-71.10977366,250173538002,42.3694630,-71.10833580,mixed_use,multi_family,commercial,office,20,nothing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,1.24079877e+19,EAT,HOME,2023-02-03 13:56:41,2023-02-03 13:59:21,2,0.44720119,NaN,250173512043,42.379771,-71.10544241,250173512034,42.3759925,-71.10314558,residential,multi_family,commercial,retail,20,nothing
239,1.37504330e+19,EAT,HOME,2023-02-03 08:11:33,2023-02-03 08:12:56,1,0.49044096,NaN,250173512043,42.379771,-71.10544241,250173512034,42.3759925,-71.10314558,residential,multi_family,commercial,retail,20,nothing
240,1.72068329e+19,EAT,HOME,2023-02-03 14:00:16,2023-02-03 14:01:44,1,0.33963478,NaN,250173512043,42.379771,-71.10544241,250173512034,42.3759925,-71.10314558,residential,multi_family,commercial,retail,20,nothing
241,5.44121616e+16,EAT,HOME,2023-02-03 20:45:44,2023-02-03 20:50:50,5,1.42958277,NaN,250173537002,42.375375,-71.11070189,250173538002,42.3694630,-71.10833580,residential,multi_family,commercial,office,20,nothing


In [180]:
# There is one trip that is done when all restaurants are closed so ignore
nonassigned_df.drop(nonassigned_df.loc[nonassigned_df['placekey']=='nothing'].index, inplace=True)
nonassigned_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey


In [181]:
# Add latitude and longitude of every assigned restaurant
na_df = nonassigned_df.merge(filtered_df[['placekey','latitude', 'longitude']], how='left', left_on='placekey', right_on='placekey')
na_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,latitude,longitude


# MERGE TRIPS

In [182]:
# Merge the assigned with the non assigned
d_list=['nothing']
eat_df=eat_df[eat_df['placekey'].isin(d_list)==False]
eattrips_df=pd.concat([eat_df,na_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
eattrips_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,latitude,longitude
0,1.22674427e+19,EAT,HOME,2023-02-03 00:03:19,2023-02-03 00:05:28,2,0.11179820,NaN,250173527001,42.3731030,...,42.3697630,-71.08985094,residential,multi_family,mixed_use,multi_family,20,239-222@62j-sj2-7dv,42.367673,-71.090329
1,1.17350550e+19,EAT,HOME,2023-02-03 00:08:15,2023-02-03 00:09:07,0,0.04538256,NaN,250173537001,42.3750675,...,42.3750675,-71.10725690,residential,single_family,commercial,retail,20,25h-222@62j-sk2-p7q,42.377946,-71.107261
2,1.76322124e+19,EAT,WORK,2023-02-03 00:12:03,2023-02-03 00:31:47,19,1.02195261,NaN,250173515002,42.3768545,...,42.3668330,-71.10146531,mixed_use,multi_family,commercial,retail,20,22b-222@62j-sj2-rff,42.365028,-71.102423
3,1.38419628e+19,EAT,LODGING,2023-02-03 00:13:11,2023-02-03 00:36:32,23,1.20961137,NaN,250173537002,42.3753750,...,42.3616040,-71.09919947,commercial,retail,mixed_use,retail,20,24d-222@62j-sj2-s5z,42.363664,-71.101451
4,1.62290058e+19,EAT,EAT,2023-02-03 00:34:13,2023-02-03 00:34:32,0,0.01652482,NaN,250173522001,42.3720955,...,42.3720955,-71.08541594,mixed_use,retail,mixed_use,multi_family,20,22d-222@62j-sj2-vzz,42.371281,-71.083318


In [183]:
eattrips_df=eattrips_df.rename(columns={"latitude": "start_latitude"}, errors="raise")
eattrips_df=eattrips_df.rename(columns={"longitude": "start_longitude"}, errors="raise")
eattrips_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,1.22674427e+19,EAT,HOME,2023-02-03 00:03:19,2023-02-03 00:05:28,2,0.11179820,NaN,250173527001,42.3731030,...,42.3697630,-71.08985094,residential,multi_family,mixed_use,multi_family,20,239-222@62j-sj2-7dv,42.367673,-71.090329
1,1.17350550e+19,EAT,HOME,2023-02-03 00:08:15,2023-02-03 00:09:07,0,0.04538256,NaN,250173537001,42.3750675,...,42.3750675,-71.10725690,residential,single_family,commercial,retail,20,25h-222@62j-sk2-p7q,42.377946,-71.107261
2,1.76322124e+19,EAT,WORK,2023-02-03 00:12:03,2023-02-03 00:31:47,19,1.02195261,NaN,250173515002,42.3768545,...,42.3668330,-71.10146531,mixed_use,multi_family,commercial,retail,20,22b-222@62j-sj2-rff,42.365028,-71.102423
3,1.38419628e+19,EAT,LODGING,2023-02-03 00:13:11,2023-02-03 00:36:32,23,1.20961137,NaN,250173537002,42.3753750,...,42.3616040,-71.09919947,commercial,retail,mixed_use,retail,20,24d-222@62j-sj2-s5z,42.363664,-71.101451
4,1.62290058e+19,EAT,EAT,2023-02-03 00:34:13,2023-02-03 00:34:32,0,0.01652482,NaN,250173522001,42.3720955,...,42.3720955,-71.08541594,mixed_use,retail,mixed_use,multi_family,20,22d-222@62j-sj2-vzz,42.371281,-71.083318


In [184]:
len(eattrips_df)

7744

# OSM DATA

In [185]:
# Import CAMBRIDGE Open Street Map Data to be able to assign exact location to the origin of the trips: 
# Transportation Utilities, Civic Institutional Buildings
filepath_osm_tangible_table = ".././02 Data/OSM/Tangible Table/"
osm_amenities_df = pd.read_csv(filepath_osm_tangible_table + 'tangibletableamenitypois.csv')
osm_buildings_df = pd.read_csv(filepath_osm_tangible_table + 'tangibletablebuildingpois.csv')

# SAFEGRAPH DATA

In [186]:
safegraph_df = places_df.merge(patterns_df[['placekey','poi_cbg']], how='left', left_on='placekey', right_on='placekey')
drop_list = ['open_hours','location_name','index']
safegraph_df = safegraph_df.reset_index().drop(drop_list,axis=1)
safegraph_df.head()

,placekey,latitude,longitude,top_category,poi_cbg
0,222-222@62j-sj2-sqz,42.365577,-71.105894,Religious Organizations,2.50173535e+11
1,223-222@62j-sj2-hh5,42.370093,-71.102315,Other Amusement and Recreation Industries,2.50173529e+11
2,224-222@62j-sj2-kcq,42.373707,-71.104915,Lessors of Real Estate,NaN
3,226-222@62j-sj2-7bk,42.369144,-71.090895,Lessors of Real Estate,NaN
4,226-222@62j-sj2-ks5,42.371825,-71.101943,Lessors of Real Estate,NaN


# TRIP ORIGIN STUDY

In [187]:
eattrips_df_copy=eattrips_df.copy()

## CIVIC INSTITUTIONAL 

In [188]:
# Filter all the CIVIC INSTITUTIONAL buildings (based on origin land use 1)
replica_ci_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='civic_institutional'].reset_index().drop('index',axis=1)

In [189]:
# Drop CIVIC INSTITUTIONAL trips to check all of them assigned at the end
drop_list=['civic_institutional']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False].reset_index().drop('index', axis=1)

In [190]:
# Split all CIVIC INSTITUTIONAL trips by the existing origin land use 2 values
replica_education_ci_df=replica_ci_df[replica_ci_df['origin_land_use_l2']=='education'].reset_index().drop('index',axis=1)
replica_healthcare_ci_df=replica_ci_df[replica_ci_df['origin_land_use_l2']=='healthcare'].reset_index().drop('index',axis=1)
replica_civic_institutional_ci_df=replica_ci_df[replica_ci_df['origin_land_use_l2']=='civic_institutional'].reset_index().drop('index',axis=1)

## COMMERCIAL 

In [191]:
# Filter all the COMMERCIAL buildings (based on origin land use 1)
replica_commercial_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='commercial'].reset_index().drop('index',axis=1)

In [192]:
# Drop COMMERCIAL trips to check all of them assigned at the end
drop_list=['commercial']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False].reset_index().drop('index', axis=1)

In [193]:
# Split all COMMERCIAL trips by the existing origin land use 2 values
replica_office_c_df=replica_commercial_df[replica_commercial_df['origin_land_use_l2']=='office'].reset_index().drop('index',axis=1)
replica_retail_c_df=replica_commercial_df[replica_commercial_df['origin_land_use_l2']=='retail'].reset_index().drop('index',axis=1)
replica_non_retail_c_df=replica_commercial_df[replica_commercial_df['origin_land_use_l2']=='non_retail_attraction'].reset_index().drop('index',axis=1)

In [194]:
# Split all origin land use 2 = retail by previous activity type
restaurants_c_list=['EAT', 'SOCIAL','OTHER_ACTIVITY_TYPE']
retail_restaurants_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type'].isin(restaurants_c_list)].reset_index().drop('index',axis=1)
retail_shopping_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type']=='SHOP'].reset_index().drop('index',axis=1)
work_c_list=['WORK','MAINTENANCE']
retail_work_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type'].isin(work_c_list)].reset_index().drop('index',axis=1)
retail_park_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type']=='RECREATION'].reset_index().drop('index',axis=1)
retail_education_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type']=='SCHOOL'].reset_index().drop('index',axis=1)
retail_hotel_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type']=='LODGING'].reset_index().drop('index',axis=1)

## MIXED USE

In [195]:
# Filter all the MIXED USE buildings (based on origin land use 1)
replica_mixed_use_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='mixed_use'].reset_index().drop('index',axis=1)

In [196]:
# Drop MIXED USE trips to check all of them assigned at the end
drop_list=['mixed_use']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False].reset_index().drop('index', axis=1)

In [197]:
# Split all MIXED USE trips by the existing origin land use 2 values
replica_industrial_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='industrial'].reset_index().drop('index',axis=1)
replica_retail_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='retail'].reset_index().drop('index',axis=1)
replica_non_retail_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='non_retail_attraction'].reset_index().drop('index',axis=1)
replica_civic_institutional_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='civic_institutional'].reset_index().drop('index',axis=1)
residential_list=['multi_family','single_family']
replica_residential_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2'].isin(residential_list)].reset_index().drop('index',axis=1)
replica_education_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='education'].reset_index().drop('index',axis=1)
replica_office_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='office'].reset_index().drop('index',axis=1)
replica_healthcare_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='healthcare'].reset_index().drop('index',axis=1)
replica_park_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='open_space'].reset_index().drop('index',axis=1)
replica_tu_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='transportation_utilities'].reset_index().drop('index',axis=1)

In [198]:
# Split all origin land use 2 = retail by previous activity type
residential_mu_list=['HOME','WORK_FROM_HOME']
retail_residential_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type'].isin(residential_mu_list)].reset_index().drop('index',axis=1)
restaurants_mu_list=['EAT', 'SOCIAL','OTHER_ACTIVITY_TYPE']
retail_restaurants_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type'].isin(restaurants_mu_list)].reset_index().drop('index',axis=1)
retail_shopping_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type']=='SHOP'].reset_index().drop('index',axis=1)
work_mu_list=['WORK','MAINTENANCE']
retail_work_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type'].isin(work_mu_list)].reset_index().drop('index',axis=1)
retail_park_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type']=='RECREATION'].reset_index().drop('index',axis=1)
retail_education_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type']=='SCHOOL'].reset_index().drop('index',axis=1)

## RESIDENTIAL 

In [199]:
# Filter all the RESIDENTIAL use buildings (based on origin land use 1)
replica_residential_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='residential'].reset_index().drop('index',axis=1)

In [200]:
# Drop RESIDENTIAL use trips to check all of them assigned at the end
drop_list=['residential']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False].reset_index().drop('index', axis=1)

In [201]:
# Merge de residential use buildings with the mixed use ones with origin land use 2 = residential and  origin land use 2 = retail with previous activity HOME / WORK FROM HOME
replica_residential_df=pd.concat([replica_residential_df,replica_residential_mu_df,retail_residential_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_residential_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,1.22674427e+19,EAT,HOME,2023-02-03 00:03:19,2023-02-03 00:05:28,2,0.11179820,NaN,250173527001,42.3731030,...,42.3697630,-71.08985094,residential,multi_family,mixed_use,multi_family,20,239-222@62j-sj2-7dv,42.367673,-71.090329
1,1.17350550e+19,EAT,HOME,2023-02-03 00:08:15,2023-02-03 00:09:07,0,0.04538256,NaN,250173537001,42.3750675,...,42.3750675,-71.10725690,residential,single_family,commercial,retail,20,25h-222@62j-sk2-p7q,42.377946,-71.107261
2,1.76322124e+19,EAT,WORK,2023-02-03 00:12:03,2023-02-03 00:31:47,19,1.02195261,NaN,250173515002,42.3768545,...,42.3668330,-71.10146531,mixed_use,multi_family,commercial,retail,20,22b-222@62j-sj2-rff,42.365028,-71.102423
3,1.67153631e+19,EAT,EAT,2023-02-03 00:45:13,2023-02-03 00:45:32,0,0.01652482,NaN,250173522001,42.3720955,...,42.3720955,-71.08541594,mixed_use,multi_family,mixed_use,retail,20,23d-222@62j-sj2-xbk,42.371654,-71.086311
4,2.74441125e+18,EAT,SOCIAL,2023-02-03 00:47:20,2023-02-03 00:52:15,4,0.25483028,NaN,250173531012,42.3633570,...,42.3633570,-71.09849179,residential,multi_family,commercial,retail,20,227-222@62j-sj2-bx5,42.361745,-71.097077


In [202]:
# Load the RESIDENTIAL shapefile data
filepath_residential_csv = ".././02 Data/Shapefile/Tangible-Table/"
residential_df = pd.read_csv(filepath_residential_csv + 'tangibletable_residentialbuildings.csv')
residential_df.head()

,Name,Shape_Leng,Shape_Area,lon_POI,lat_POI,usage,geometry
0,99 INMAN ST,265.39919902,3430.31518555,-71.10191137,42.37193600,R,"POLYGON ((-71.1017215158527 42.37191554184239,..."
1,3-5 BOW ST,344.37320593,4781.27392578,-71.11594241,42.37200179,R,POLYGON ((-71.11573597081214 42.37192616650645...
2,686 CAMBRIDGE ST,210.97249614,2766.54516602,-71.08804768,42.37190903,R,POLYGON ((-71.08793079324698 42.37196849805142...
3,84 WILLOW ST,184.30621878,1587.43139648,-71.09238795,42.37193311,R,"POLYGON ((-71.09226384439502 42.3719318003, -7..."
4,26 SEVENTH ST,127.86338193,799.13598633,-71.08534646,42.37194092,R,POLYGON ((-71.08525595720681 42.37195271081824...


In [203]:
# Based on the trip origin lat,lon assign the closest RESIDENTIAL building with function KDTree
pd.set_option("display.precision", 8)
replica_residential_df['end_latitude']=''
replica_residential_df['end_longitude']=''
Z = np.array(residential_df[['lat_POI', 'lon_POI']])
tree = spatial.KDTree(Z)
for i in range(len(replica_residential_df)):
    dist, idxs = tree.query(np.array(replica_residential_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 20)
    dist_selected = np.random.choice(dist, 1)
    for j in range(20):
        if dist[j] == dist_selected:
            idx_selected = idxs[j]
            break
    replica_residential_df.at[i, 'end_latitude'] = residential_df.at[idx_selected,'lat_POI']
    replica_residential_df.at[i, 'end_longitude'] = residential_df.at[idx_selected,'lon_POI']
replica_residential_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.22674427e+19,EAT,HOME,2023-02-03 00:03:19,2023-02-03 00:05:28,2,0.11179820,NaN,250173527001,42.3731030,...,residential,multi_family,mixed_use,multi_family,20,239-222@62j-sj2-7dv,42.367673,-71.090329,42.373454,-71.089322
1,1.17350550e+19,EAT,HOME,2023-02-03 00:08:15,2023-02-03 00:09:07,0,0.04538256,NaN,250173537001,42.3750675,...,residential,single_family,commercial,retail,20,25h-222@62j-sk2-p7q,42.377946,-71.107261,42.373983,-71.107175
2,1.76322124e+19,EAT,WORK,2023-02-03 00:12:03,2023-02-03 00:31:47,19,1.02195261,NaN,250173515002,42.3768545,...,mixed_use,multi_family,commercial,retail,20,22b-222@62j-sj2-rff,42.365028,-71.102423,42.377015,-71.092825
3,1.67153631e+19,EAT,EAT,2023-02-03 00:45:13,2023-02-03 00:45:32,0,0.01652482,NaN,250173522001,42.3720955,...,mixed_use,multi_family,mixed_use,retail,20,23d-222@62j-sj2-xbk,42.371654,-71.086311,42.371964,-71.085711
4,2.74441125e+18,EAT,SOCIAL,2023-02-03 00:47:20,2023-02-03 00:52:15,4,0.25483028,NaN,250173531012,42.3633570,...,residential,multi_family,commercial,retail,20,227-222@62j-sj2-bx5,42.361745,-71.097077,42.364506,-71.097976


In [204]:
# Save the relevant data for the GAMA simulation model
residential_trips_df=replica_residential_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                            'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
residential_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.22674427e+19,42.367673,-71.090329,42.373454,-71.089322,EAT,2023-02-03 00:03:19,2023-02-03 00:05:28,2,0.11179820,20
1,1.17350550e+19,42.377946,-71.107261,42.373983,-71.107175,EAT,2023-02-03 00:08:15,2023-02-03 00:09:07,0,0.04538256,20
2,1.76322124e+19,42.365028,-71.102423,42.377015,-71.092825,EAT,2023-02-03 00:12:03,2023-02-03 00:31:47,19,1.02195261,20
3,1.67153631e+19,42.371654,-71.086311,42.371964,-71.085711,EAT,2023-02-03 00:45:13,2023-02-03 00:45:32,0,0.01652482,20
4,2.74441125e+18,42.361745,-71.097077,42.364506,-71.097976,EAT,2023-02-03 00:47:20,2023-02-03 00:52:15,4,0.25483028,20


## INDUSTRIAL

In [205]:
# Filter all the INDUSTRIAL use buildings (based on origin land use 1)
replica_industrial_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='industrial'].reset_index().drop('index',axis=1)

In [206]:
# Drop INDUSTRIAL use trips to check if all of them are assigned at the end
drop_list=['industrial']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False].reset_index().drop('index', axis=1)

In [207]:
# Merge the INDUSTRIAL use buildings with the mixed use ones with origin land use 2 = industrial
replica_industrial_df=pd.concat([replica_industrial_df,replica_industrial_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_industrial_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,6.67862780e+18,EAT,RECREATION,2023-02-03 05:12:26,2023-02-03 05:14:02,1,0.52709554,NaN,250173523001,42.3675710,...,42.3585020,-71.08549670,mixed_use,industrial,civic_institutional,education,20,zzw-222@62j-sj2-b49,42.359142,-71.092973
1,3.95862208e+18,EAT,WORK,2023-02-03 06:17:05,2023-02-03 06:19:12,2,0.11027069,NaN,250173531011,42.3616040,...,42.3633570,-71.09849179,industrial,industrial,civic_institutional,education,20,22x-222@62j-sj2-ffz,42.363390,-71.096870
2,1.44601753e+19,EAT,WORK,2023-02-03 06:19:11,2023-02-03 06:26:08,6,0.36016455,NaN,250173531011,42.3616040,...,42.3616040,-71.09919947,industrial,industrial,commercial,retail,20,24d-222@62j-sj2-s5z,42.363664,-71.101451
3,9.81099866e+18,EAT,WORK,2023-02-03 07:13:32,2023-02-03 07:34:28,20,1.49951664,NaN,250173532003,42.3553975,...,42.3715445,-71.10118533,industrial,industrial,commercial,retail,20,222-223@62j-sj3-9fz,42.372775,-71.099329
4,1.10963563e+18,EAT,RECREATION,2023-02-03 07:39:28,2023-02-03 07:41:04,1,0.52709554,NaN,250173523001,42.3675710,...,42.3585020,-71.08549670,mixed_use,industrial,civic_institutional,education,20,zzy-222@62j-sj4-v9f,42.361317,-71.085981


In [208]:
# SafeGraph Data for industrial buildings
industrial_poi_list=['Other Amusement and Recreation Industries','Other Miscellaneous Manufacturing','Gambling Industries',
                    'Coating, Engraving, Heat Treating, and Allied Activities','Machinery, Equipment, and Supplies Merchant Wholesalers',
                    'Motion Picture and Video Industries','Converted Paper Product Manufacturing','Glass and Glass Product Manufacturing',
                    'Electric Power Generation, Transmission and Distribution','Beverage Manufacturing', 'Sound Recording Industries',
                    'Bakeries and Tortilla Manufacturing']
industrial_sg_df = safegraph_df[places_df['top_category'].isin(industrial_poi_list)]
industrial_sg_df = industrial_sg_df.reset_index().drop('index',axis=1)
industrial_sg_df.head()

,placekey,latitude,longitude,top_category,poi_cbg
0,223-222@62j-sj2-hh5,42.370093,-71.102315,Other Amusement and Recreation Industries,2.50173529e+11
1,235-222@62j-sj2-835,42.371124,-71.097079,Other Amusement and Recreation Industries,NaN
2,229-222@62j-sj2-hh5,42.369975,-71.102064,Other Amusement and Recreation Industries,2.50173529e+11
3,zzw-229@62j-sj2-rkz,42.364956,-71.103455,Other Amusement and Recreation Industries,2.50173530e+11
4,zzy-223@62j-sjt-kxq,42.373029,-71.122372,Other Amusement and Recreation Industries,2.50173541e+11


In [209]:
# Based on the trip origin lat,lon assign the closest INDUSTRIAL building with function KDTree
pd.set_option("display.precision", 8)
replica_industrial_df['end_latitude']=''
replica_industrial_df['end_longitude']=''
for i in range(len(replica_industrial_df)):
    blockgroup = replica_industrial_df.at[i,'origin_bgrp']
    industrial_bgrp_sg_df = industrial_sg_df[industrial_sg_df['poi_cbg']==blockgroup]
    industrial_bgrp_sg_df = industrial_bgrp_sg_df.reset_index().drop('index',axis=1)
    if industrial_bgrp_sg_df.empty == False:
        placekey = industrial_bgrp_sg_df['placekey']
        placekey_selected = np.random.choice(placekey, 1)
        for j in range(len(industrial_bgrp_sg_df)):
            if industrial_bgrp_sg_df.at[j,'placekey'] == placekey_selected:
                replica_industrial_df.at[i, 'end_latitude'] = industrial_bgrp_sg_df.at[j,'latitude']
                replica_industrial_df.at[i, 'end_longitude'] = industrial_bgrp_sg_df.at[j,'longitude']
                break
    else:
        Z = np.array(industrial_sg_df[['latitude', 'longitude']])
        tree = spatial.KDTree(Z)
        dist, idxs = tree.query(np.array(replica_industrial_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 1)
        replica_industrial_df.at[i, 'end_latitude'] = industrial_sg_df.at[idxs,'latitude']
        replica_industrial_df.at[i, 'end_longitude'] = industrial_sg_df.at[idxs,'longitude']
replica_industrial_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,6.67862780e+18,EAT,RECREATION,2023-02-03 05:12:26,2023-02-03 05:14:02,1,0.52709554,NaN,250173523001,42.3675710,...,mixed_use,industrial,civic_institutional,education,20,zzw-222@62j-sj2-b49,42.359142,-71.092973,42.36864,-71.08649
1,3.95862208e+18,EAT,WORK,2023-02-03 06:17:05,2023-02-03 06:19:12,2,0.11027069,NaN,250173531011,42.3616040,...,industrial,industrial,civic_institutional,education,20,22x-222@62j-sj2-ffz,42.363390,-71.096870,42.360789,-71.096565
2,1.44601753e+19,EAT,WORK,2023-02-03 06:19:11,2023-02-03 06:26:08,6,0.36016455,NaN,250173531011,42.3616040,...,industrial,industrial,commercial,retail,20,24d-222@62j-sj2-s5z,42.363664,-71.101451,42.363341,-71.101265
3,9.81099866e+18,EAT,WORK,2023-02-03 07:13:32,2023-02-03 07:34:28,20,1.49951664,NaN,250173532003,42.3553975,...,industrial,industrial,commercial,retail,20,222-223@62j-sj3-9fz,42.372775,-71.099329,42.357233,-71.106431
4,1.10963563e+18,EAT,RECREATION,2023-02-03 07:39:28,2023-02-03 07:41:04,1,0.52709554,NaN,250173523001,42.3675710,...,mixed_use,industrial,civic_institutional,education,20,zzy-222@62j-sj4-v9f,42.361317,-71.085981,42.365003,-71.087029


In [210]:
# Save the relevant data for the GAMA simulation model
industrial_trips_df=replica_industrial_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                           'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
industrial_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,6.67862780e+18,42.359142,-71.092973,42.36864,-71.08649,EAT,2023-02-03 05:12:26,2023-02-03 05:14:02,1,0.52709554,20
1,3.95862208e+18,42.363390,-71.096870,42.360789,-71.096565,EAT,2023-02-03 06:17:05,2023-02-03 06:19:12,2,0.11027069,20
2,1.44601753e+19,42.363664,-71.101451,42.363341,-71.101265,EAT,2023-02-03 06:19:11,2023-02-03 06:26:08,6,0.36016455,20
3,9.81099866e+18,42.372775,-71.099329,42.357233,-71.106431,EAT,2023-02-03 07:13:32,2023-02-03 07:34:28,20,1.49951664,20
4,1.10963563e+18,42.361317,-71.085981,42.365003,-71.087029,EAT,2023-02-03 07:39:28,2023-02-03 07:41:04,1,0.52709554,20


## OFFICE 

In [211]:
# Merge de OFFICE use buildings: mixed use and commercial ones with origin land use 2 = office
replica_office_df=pd.concat([replica_office_mu_df,replica_office_c_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_office_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,1.03393589e+19,EAT,MAINTENANCE,2023-02-03 06:09:44,2023-02-03 06:10:43,0,0.05117650,NaN,250173531012,42.363357,...,42.363357,-71.09849179,mixed_use,office,mixed_use,office,20,227-222@62j-sj2-bx5,42.361745,-71.097077
1,1.45081541e+19,EAT,HOME,2023-02-03 06:12:48,2023-02-03 06:14:24,1,0.08306649,NaN,250173538002,42.369463,...,42.368343,-71.10977366,mixed_use,office,mixed_use,multi_family,20,27g-223@62j-sj2-jqf,42.368689,-71.109924
2,1.09566229e+19,EAT,MAINTENANCE,2023-02-03 06:29:57,2023-02-03 06:30:30,0,0.02916992,NaN,250173537002,42.375375,...,42.373904,-71.11366683,mixed_use,office,commercial,retail,20,zzw-222@62j-sjv-cnq,42.375586,-71.113753
3,1.40183764e+19,EAT,EAT,2023-02-03 06:30:42,2023-02-03 06:31:13,0,0.02729844,NaN,250173537004,42.372157,...,42.372157,-71.11732789,commercial,office,civic_institutional,education,20,23f-222@62j-sjv-b6k,42.372895,-71.117919
4,4.62105066e+18,EAT,HOME,2023-02-03 06:42:23,2023-02-03 06:45:07,2,0.14183204,NaN,250173531011,42.361604,...,42.361604,-71.09919947,mixed_use,office,commercial,retail,20,24d-222@62j-sj2-s5z,42.363664,-71.101451


In [212]:
# SafeGraph Data for OFFICE buildings
office_poi_list=['Offices of Real Estate Agents and Brokers','Management of Companies and Enterprises','Agencies, Brokerages, and Other Insurance Related Activities',
                    'Electronic and Precision Equipment Repair and Maintenance','Architectural, Engineering, and Related Services',
                    'Personal and Household Goods Repair and Maintenance','Other Professional, Scientific, and Technical Services',
                    'Building Equipment Contractors','Automobile Dealers','Activities Related to Real Estate','Other Financial Investment Activities',
                    'Travel Arrangement and Reservation Services','Radio and Television Broadcasting','Automotive Equipment Rental and Leasing',
                    'Management, Scientific, and Technical Consulting Services','Building Material and Supplies Dealers',
                    'Consumer Goods Rental','Building Finishing Contractors','Couriers and Express Delivery Services',
                    'Cable and Other Subscription Programming','Advertising, Public Relations, and Related Services',
                    'Administration of Human Resource Programs','Other Specialty Trade Contractors']
office_sg_df = safegraph_df[safegraph_df['top_category'].isin(office_poi_list)]
office_sg_df = office_sg_df.reset_index().drop('index',axis=1)
office_sg_df.head()

,placekey,latitude,longitude,top_category,poi_cbg
0,23m-222@62j-sj8-q4v,42.379402,-71.091611,Building Finishing Contractors,2.50173515e+11
1,zzy-223@62j-sj2-89z,42.372900,-71.095861,"Other Professional, Scientific, and Technical ...",NaN
2,22q-222@62j-sj2-rkz,42.366060,-71.103222,Personal and Household Goods Repair and Mainte...,NaN
3,222-222@62j-sj2-v4v,42.374443,-71.087895,Building Finishing Contractors,2.50173515e+11
4,zzw-228@62j-sj2-t5f,42.367465,-71.107366,Offices of Real Estate Agents and Brokers,NaN


In [213]:
# Based on the trip origin lat,lon assign the closest OFFICE building with function KDTree
pd.set_option("display.precision", 8)
replica_office_df['end_latitude']=''
replica_office_df['end_longitude']=''
for i in range(len(replica_office_df)):
    blockgroup = replica_office_df.at[i,'origin_bgrp']
    office_bgrp_sg_df = office_sg_df[office_sg_df['poi_cbg']==blockgroup]
    office_bgrp_sg_df = office_bgrp_sg_df.reset_index().drop('index',axis=1)
    if office_bgrp_sg_df.empty == False:
        placekey = office_bgrp_sg_df['placekey']
        placekey_selected = np.random.choice(placekey, 1)
        for j in range(len(office_bgrp_sg_df)):
            if office_bgrp_sg_df.at[j,'placekey'] == placekey_selected:
                replica_office_df.at[i, 'end_latitude'] = office_bgrp_sg_df.at[j,'latitude']
                replica_office_df.at[i, 'end_longitude'] = office_bgrp_sg_df.at[j,'longitude']
                break
    else:
        Z = np.array(office_sg_df[['latitude', 'longitude']])
        tree = spatial.KDTree(Z)
        dist, idxs = tree.query(np.array(replica_office_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 1)
        replica_office_df.at[i, 'end_latitude'] = office_sg_df.at[idxs,'latitude']
        replica_office_df.at[i, 'end_longitude'] = office_sg_df.at[idxs,'longitude']
replica_office_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.03393589e+19,EAT,MAINTENANCE,2023-02-03 06:09:44,2023-02-03 06:10:43,0,0.05117650,NaN,250173531012,42.363357,...,mixed_use,office,mixed_use,office,20,227-222@62j-sj2-bx5,42.361745,-71.097077,42.363096,-71.098512
1,1.45081541e+19,EAT,HOME,2023-02-03 06:12:48,2023-02-03 06:14:24,1,0.08306649,NaN,250173538002,42.369463,...,mixed_use,office,mixed_use,multi_family,20,27g-223@62j-sj2-jqf,42.368689,-71.109924,42.36972,-71.106284
2,1.09566229e+19,EAT,MAINTENANCE,2023-02-03 06:29:57,2023-02-03 06:30:30,0,0.02916992,NaN,250173537002,42.375375,...,mixed_use,office,commercial,retail,20,zzw-222@62j-sjv-cnq,42.375586,-71.113753,42.373614,-71.109717
3,1.40183764e+19,EAT,EAT,2023-02-03 06:30:42,2023-02-03 06:31:13,0,0.02729844,NaN,250173537004,42.372157,...,commercial,office,civic_institutional,education,20,23f-222@62j-sjv-b6k,42.372895,-71.117919,42.37275,-71.11741
4,4.62105066e+18,EAT,HOME,2023-02-03 06:42:23,2023-02-03 06:45:07,2,0.14183204,NaN,250173531011,42.361604,...,mixed_use,office,commercial,retail,20,24d-222@62j-sj2-s5z,42.363664,-71.101451,42.362525,-71.099602


In [214]:
# Save the relevant data for the GAMA simulation model
office_trips_df=replica_office_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                   'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
office_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.03393589e+19,42.361745,-71.097077,42.363096,-71.098512,EAT,2023-02-03 06:09:44,2023-02-03 06:10:43,0,0.05117650,20
1,1.45081541e+19,42.368689,-71.109924,42.36972,-71.106284,EAT,2023-02-03 06:12:48,2023-02-03 06:14:24,1,0.08306649,20
2,1.09566229e+19,42.375586,-71.113753,42.373614,-71.109717,EAT,2023-02-03 06:29:57,2023-02-03 06:30:30,0,0.02916992,20
3,1.40183764e+19,42.372895,-71.117919,42.37275,-71.11741,EAT,2023-02-03 06:30:42,2023-02-03 06:31:13,0,0.02729844,20
4,4.62105066e+18,42.363664,-71.101451,42.362525,-71.099602,EAT,2023-02-03 06:42:23,2023-02-03 06:45:07,2,0.14183204,20


## SHOP 

In [215]:
# Merge de SHOP use buildings: mixed use and commercial ones with origin land use 2 = retail and with previous activity SHOP
replica_shop_df=pd.concat([retail_shopping_c_df,retail_shopping_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_shop_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,1.80545723e+19,EAT,SHOP,2023-02-03 01:30:10,2023-02-03 01:57:46,27,1.41427884,NaN,250173540003,42.3766435,...,42.3683430,-71.10977366,commercial,retail,commercial,retail,20,27g-223@62j-sj2-jqf,42.368689,-71.109924
1,1.05025476e+19,EAT,SHOP,2023-02-03 03:16:46,2023-02-03 03:38:20,21,1.11695665,NaN,250173537001,42.3750675,...,42.3616040,-71.09919947,commercial,retail,mixed_use,retail,20,23q-222@62j-sj2-p35,42.364217,-71.102907
2,9.21740289e+17,EAT,SHOP,2023-02-03 04:02:08,2023-02-03 04:03:56,1,0.09345651,NaN,250173515002,42.3768545,...,42.3768545,-71.09241285,commercial,retail,commercial,retail,20,zzw-223@62j-sj3-b8v,42.379303,-71.095686
3,7.05405313e+18,EAT,SHOP,2023-02-03 06:28:15,2023-02-03 06:31:57,3,1.39050595,NaN,250173530002,42.3668330,...,42.3721570,-71.11732789,commercial,retail,civic_institutional,education,20,22d-222@62j-sjv-8qf,42.372303,-71.116061
4,1.53765441e+19,EAT,SHOP,2023-02-03 06:33:07,2023-02-03 06:47:58,14,1.88275453,NaN,250173537004,42.3721570,...,42.3720955,-71.08541594,commercial,retail,mixed_use,multi_family,20,22d-222@62j-sj2-vzz,42.371281,-71.083318


In [216]:
# SafeGraph Data for SHOPPING buildings
shop_poi_list=['Other Miscellaneous Store Retailers','Jewelry, Luggage, and Leather Goods Stores','Clothing Stores',
              'Office Supplies, Stationery, and Gift Stores','Furniture Stores','Beer, Wine, and Liquor Stores','Grocery Stores',
              'Specialty Food Stores','Shoe Stores','Florists','Health and Personal Care Stores','Printing and Related Support Activities',
              'Home Furnishings Stores','Electronics and Appliance Stores','Department Stores','Used Merchandise Stores',
              "Drugs and Druggists' Sundries Merchant Wholesalers",'General Merchandise Stores, including Warehouse Clubs and Supercenters',
              'Lawn and Garden Equipment and Supplies Stores','Other Motor Vehicle Dealers','Book Stores and News Dealers']
shop_sg_df = safegraph_df[safegraph_df['top_category'].isin(shop_poi_list)]
shop_sg_df = shop_sg_df.reset_index().drop('index',axis=1)
shop_sg_df.head()

,placekey,latitude,longitude,top_category,poi_cbg
0,24d-222@62j-sj2-nyv,42.362727,-71.105040,Grocery Stores,NaN
1,222-222@62j-sj2-dn5,42.360119,-71.088622,Specialty Food Stores,2.50173531e+11
2,228-222@62j-sj2-k4v,42.374851,-71.104384,Health and Personal Care Stores,NaN
3,229-222@62j-sjv-975,42.374047,-71.121837,Clothing Stores,NaN
4,22k-222@62j-sjv-ndv,42.373857,-71.124783,Electronics and Appliance Stores,2.50173541e+11


In [217]:
# Based on the trip origin lat,lon assign the closest SHOPPING building with function KDTree
pd.set_option("display.precision", 8)
replica_shop_df['end_latitude']=''
replica_shop_df['end_longitude']=''
for i in range(len(replica_shop_df)):
    blockgroup = replica_shop_df.at[i,'origin_bgrp']
    shop_bgrp_sg_df = shop_sg_df[shop_sg_df['poi_cbg']==blockgroup]
    shop_bgrp_sg_df = shop_bgrp_sg_df.reset_index().drop('index',axis=1)
    if shop_bgrp_sg_df.empty == False:
        placekey = shop_bgrp_sg_df['placekey']
        placekey_selected = np.random.choice(placekey, 1)
        for j in range(len(shop_bgrp_sg_df)):
            if shop_bgrp_sg_df.at[j,'placekey'] == placekey_selected:
                replica_shop_df.at[i, 'end_latitude'] = shop_bgrp_sg_df.at[j,'latitude']
                replica_shop_df.at[i, 'end_longitude'] = shop_bgrp_sg_df.at[j,'longitude']
                break
    else:
        Z = np.array(shop_sg_df[['latitude', 'longitude']])
        tree = spatial.KDTree(Z)
        dist, idxs = tree.query(np.array(replica_shop_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 1)
        replica_shop_df.at[i, 'end_latitude'] = shop_sg_df.at[idxs,'latitude']
        replica_shop_df.at[i, 'end_longitude'] = shop_sg_df.at[idxs,'longitude']
replica_shop_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.80545723e+19,EAT,SHOP,2023-02-03 01:30:10,2023-02-03 01:57:46,27,1.41427884,NaN,250173540003,42.3766435,...,commercial,retail,commercial,retail,20,27g-223@62j-sj2-jqf,42.368689,-71.109924,42.373571,-71.119885
1,1.05025476e+19,EAT,SHOP,2023-02-03 03:16:46,2023-02-03 03:38:20,21,1.11695665,NaN,250173537001,42.3750675,...,commercial,retail,mixed_use,retail,20,23q-222@62j-sj2-p35,42.364217,-71.102907,42.377849,-71.107487
2,9.21740289e+17,EAT,SHOP,2023-02-03 04:02:08,2023-02-03 04:03:56,1,0.09345651,NaN,250173515002,42.3768545,...,commercial,retail,commercial,retail,20,zzw-223@62j-sj3-b8v,42.379303,-71.095686,42.375492,-71.09301
3,7.05405313e+18,EAT,SHOP,2023-02-03 06:28:15,2023-02-03 06:31:57,3,1.39050595,NaN,250173530002,42.3668330,...,commercial,retail,civic_institutional,education,20,22d-222@62j-sjv-8qf,42.372303,-71.116061,42.365693,-71.103564
4,1.53765441e+19,EAT,SHOP,2023-02-03 06:33:07,2023-02-03 06:47:58,14,1.88275453,NaN,250173537004,42.3721570,...,commercial,retail,mixed_use,multi_family,20,22d-222@62j-sj2-vzz,42.371281,-71.083318,42.372884,-71.11773


In [218]:
# Save the relevant data for the GAMA simulation model
shopping_trips_df=replica_shop_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                   'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
shopping_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.80545723e+19,42.368689,-71.109924,42.373571,-71.119885,EAT,2023-02-03 01:30:10,2023-02-03 01:57:46,27,1.41427884,20
1,1.05025476e+19,42.364217,-71.102907,42.377849,-71.107487,EAT,2023-02-03 03:16:46,2023-02-03 03:38:20,21,1.11695665,20
2,9.21740289e+17,42.379303,-71.095686,42.375492,-71.09301,EAT,2023-02-03 04:02:08,2023-02-03 04:03:56,1,0.09345651,20
3,7.05405313e+18,42.372303,-71.116061,42.365693,-71.103564,EAT,2023-02-03 06:28:15,2023-02-03 06:31:57,3,1.39050595,20
4,1.53765441e+19,42.371281,-71.083318,42.372884,-71.11773,EAT,2023-02-03 06:33:07,2023-02-03 06:47:58,14,1.88275453,20


### HOTELS

In [219]:
# Merge HOTEL use trips: commercial ones with origin land use 2 = retail and with previous activity LODGING
replica_hotel_df=retail_hotel_c_df
replica_hotel_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,1.38419628e+19,EAT,LODGING,2023-02-03 00:13:11,2023-02-03 00:36:32,23,1.20961137,NaN,250173537002,42.3753750,...,42.3616040,-71.09919947,commercial,retail,mixed_use,retail,20,24d-222@62j-sj2-s5z,42.363664,-71.101451
1,1.24436353e+19,EAT,LODGING,2023-02-03 05:31:05,2023-02-03 05:35:24,4,0.22425487,NaN,250173531011,42.3616040,...,42.3616040,-71.09919947,commercial,retail,commercial,retail,20,24d-222@62j-sj2-s5z,42.363664,-71.101451
2,1.21549565e+19,EAT,LODGING,2023-02-03 05:31:05,2023-02-03 05:33:23,2,0.11954423,NaN,250173531011,42.3616040,...,42.3616040,-71.09919947,commercial,retail,commercial,non_retail_attraction,20,232-222@62j-sj2-c3q,42.361999,-71.098471
3,9.71430163e+17,EAT,LODGING,2023-02-03 07:09:39,2023-02-03 07:11:01,1,0.53826544,NaN,250173522001,42.3720955,...,42.3720955,-71.08541594,commercial,retail,mixed_use,multi_family,20,228-224@62j-sj2-x89,42.372122,-71.087128
4,1.58710300e+19,EAT,LODGING,2023-02-03 07:52:47,2023-02-03 07:54:25,1,0.54040161,NaN,250173522001,42.3720955,...,42.3720955,-71.08541594,commercial,retail,commercial,retail,20,23d-222@62j-sj2-xbk,42.371654,-71.086311


In [220]:
# SafeGraph Data for HOTEL buildings
hotel_poi_list = ['Traveler Accommodation']
hotel_sg_df = safegraph_df[safegraph_df['top_category'].isin(hotel_poi_list)]
hotel_sg_df = hotel_sg_df.reset_index().drop('index',axis=1)
hotel_sg_df.head()

,placekey,latitude,longitude,top_category,poi_cbg
0,24p-222@62j-sj2-kvf,42.370648,-71.101137,Traveler Accommodation,2.50173529e+11
1,225-225@62j-sgg-fpv,42.358602,-71.114975,Traveler Accommodation,2.50173533e+11
2,225-223@62j-sjv-qmk,42.377266,-71.123046,Traveler Accommodation,2.50173541e+11
3,zzw-224@62j-sj2-s3q,42.362183,-71.100137,Traveler Accommodation,2.50173531e+11
4,23v-222@62j-sjv-8y9,42.371098,-71.113788,Traveler Accommodation,NaN


In [221]:
# Based on the trip origin lat,lon assign the closest HOTEL building with function KDTree
pd.set_option("display.precision", 8)
replica_hotel_df['end_latitude']=''
replica_hotel_df['end_longitude']=''
for i in range(len(replica_hotel_df)):
    blockgroup = replica_hotel_df.at[i,'origin_bgrp']
    hotel_bgrp_sg_df = hotel_sg_df[hotel_sg_df['poi_cbg']==blockgroup]
    hotel_bgrp_sg_df = hotel_bgrp_sg_df.reset_index().drop('index',axis=1)
    if hotel_bgrp_sg_df.empty == False:
        placekey = hotel_bgrp_sg_df['placekey']
        placekey_selected = np.random.choice(placekey, 1)
        for j in range(len(hotel_bgrp_sg_df)):
            if hotel_bgrp_sg_df.at[j,'placekey'] == placekey_selected:
                replica_hotel_df.at[i, 'end_latitude'] = hotel_bgrp_sg_df.at[j,'latitude']
                replica_hotel_df.at[i, 'end_longitude'] = hotel_bgrp_sg_df.at[j,'longitude']
                break
    else:
        Z = np.array(hotel_sg_df[['latitude', 'longitude']])
        tree = spatial.KDTree(Z)
        dist, idxs = tree.query(np.array(replica_hotel_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 1)
        replica_hotel_df.at[i, 'end_latitude'] = hotel_sg_df.at[idxs,'latitude']
        replica_hotel_df.at[i, 'end_longitude'] = hotel_sg_df.at[idxs,'longitude']
replica_hotel_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.38419628e+19,EAT,LODGING,2023-02-03 00:13:11,2023-02-03 00:36:32,23,1.20961137,NaN,250173537002,42.3753750,...,commercial,retail,mixed_use,retail,20,24d-222@62j-sj2-s5z,42.363664,-71.101451,42.375492,-71.110654
1,1.24436353e+19,EAT,LODGING,2023-02-03 05:31:05,2023-02-03 05:35:24,4,0.22425487,NaN,250173531011,42.3616040,...,commercial,retail,commercial,retail,20,24d-222@62j-sj2-s5z,42.363664,-71.101451,42.362183,-71.100137
2,1.21549565e+19,EAT,LODGING,2023-02-03 05:31:05,2023-02-03 05:33:23,2,0.11954423,NaN,250173531011,42.3616040,...,commercial,retail,commercial,non_retail_attraction,20,232-222@62j-sj2-c3q,42.361999,-71.098471,42.362183,-71.100137
3,9.71430163e+17,EAT,LODGING,2023-02-03 07:09:39,2023-02-03 07:11:01,1,0.53826544,NaN,250173522001,42.3720955,...,commercial,retail,mixed_use,multi_family,20,228-224@62j-sj2-x89,42.372122,-71.087128,42.3639,-71.088132
4,1.58710300e+19,EAT,LODGING,2023-02-03 07:52:47,2023-02-03 07:54:25,1,0.54040161,NaN,250173522001,42.3720955,...,commercial,retail,commercial,retail,20,23d-222@62j-sj2-xbk,42.371654,-71.086311,42.3639,-71.088132


In [222]:
# Save the relevant data for the GAMA simulation model
hotel_trips_df=replica_hotel_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                 'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
hotel_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.38419628e+19,42.363664,-71.101451,42.375492,-71.110654,EAT,2023-02-03 00:13:11,2023-02-03 00:36:32,23,1.20961137,20
1,1.24436353e+19,42.363664,-71.101451,42.362183,-71.100137,EAT,2023-02-03 05:31:05,2023-02-03 05:35:24,4,0.22425487,20
2,1.21549565e+19,42.361999,-71.098471,42.362183,-71.100137,EAT,2023-02-03 05:31:05,2023-02-03 05:33:23,2,0.11954423,20
3,9.71430163e+17,42.372122,-71.087128,42.3639,-71.088132,EAT,2023-02-03 07:09:39,2023-02-03 07:11:01,1,0.53826544,20
4,1.58710300e+19,42.371654,-71.086311,42.3639,-71.088132,EAT,2023-02-03 07:52:47,2023-02-03 07:54:25,1,0.54040161,20


### RESTAURANTS

In [223]:
# Merge the RESTAURANT buildings: mixed use and commercial ones with origin land use 2 = retail with previous activity EAT, SOCIAL, OTHER_ACTIVITY_TYPE
replica_restaurants_df=pd.concat([retail_restaurants_mu_df,retail_restaurants_c_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_restaurants_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,1.62290058e+19,EAT,EAT,2023-02-03 00:34:13,2023-02-03 00:34:32,0,0.01652482,NaN,250173522001,42.3720955,...,42.3720955,-71.08541594,mixed_use,retail,mixed_use,multi_family,20,22d-222@62j-sj2-vzz,42.371281,-71.083318
1,8.46274435e+18,EAT,EAT,2023-02-03 00:47:19,2023-02-03 01:12:44,25,1.31656236,NaN,250173530002,42.3668330,...,42.3766435,-71.12102016,commercial,retail,commercial,retail,20,zzw-224@62j-sjv-835,42.374306,-71.119126
2,9.27219531e+18,EAT,EAT,2023-02-03 01:16:28,2023-02-03 01:16:28,0,0.00000000,NaN,250173540003,42.3766435,...,42.3766435,-71.12102016,commercial,retail,commercial,retail,20,23k-223@62j-sjv-99f,42.373493,-71.120642
3,2.21513782e+18,EAT,EAT,2023-02-03 05:52:44,2023-02-03 05:53:24,0,0.03484630,NaN,250173530002,42.3668330,...,42.3668330,-71.10146531,commercial,retail,commercial,retail,20,zzw-225@62j-sj2-p35,42.365034,-71.102754
4,5.38198951e+18,EAT,EAT,2023-02-03 06:05:35,2023-02-03 06:06:12,0,0.03244217,NaN,250173537004,42.3721570,...,42.3721570,-71.11732789,commercial,retail,civic_institutional,education,20,22d-222@62j-sjv-8qf,42.372303,-71.116061


In [224]:
# SafeGraph Data for RESTAURANT buildings
restaurant_poi_list = ['Restaurants and Other Eating Places','Drinking Places (Alcoholic Beverages)','Special Food Services']
restaurant_sg_df = safegraph_df[safegraph_df['top_category'].isin(restaurant_poi_list)]
restaurant_sg_df = restaurant_sg_df.reset_index().drop('index',axis=1)
restaurant_sg_df.head()

,placekey,latitude,longitude,top_category,poi_cbg
0,zzw-222@62j-sj2-b49,42.359142,-71.092973,Restaurants and Other Eating Places,2.50173531e+11
1,22h-222@62j-sj3-9zz,42.379864,-71.095129,Restaurants and Other Eating Places,2.50173513e+11
2,24h-222@62j-sj3-9j9,42.374521,-71.101502,Drinking Places (Alcoholic Beverages),2.50173512e+11
3,23s-223@62j-sjv-8n5,42.370907,-71.114488,Drinking Places (Alcoholic Beverages),NaN
4,24j-223@62j-sj2-vs5,42.371948,-71.085774,Restaurants and Other Eating Places,2.50173522e+11


In [225]:
# Based on the trip origin lat,lon assign the closest RESTAURANT building with function KDTree
pd.set_option("display.precision", 8)
replica_restaurants_df['end_latitude']=''
replica_restaurants_df['end_longitude']=''
for i in range(len(replica_restaurants_df)):
    blockgroup = replica_restaurants_df.at[i,'origin_bgrp']
    restaurant_bgrp_sg_df = restaurant_sg_df[restaurant_sg_df['poi_cbg']==blockgroup]
    restaurant_bgrp_sg_df = restaurant_bgrp_sg_df.reset_index().drop('index',axis=1)
    if restaurant_bgrp_sg_df.empty == False:
        placekey = restaurant_bgrp_sg_df['placekey']
        placekey_selected = np.random.choice(placekey, 1)
        for j in range(len(restaurant_bgrp_sg_df)):
            if restaurant_bgrp_sg_df.at[j,'placekey'] == placekey_selected:
                replica_restaurants_df.at[i, 'end_latitude'] = restaurant_bgrp_sg_df.at[j,'latitude']
                replica_restaurants_df.at[i, 'end_longitude'] = restaurant_bgrp_sg_df.at[j,'longitude']
                break
    else:
        Z = np.array(restaurant_sg_df[['latitude', 'longitude']])
        tree = spatial.KDTree(Z)
        dist, idxs = tree.query(np.array(replica_restaurants_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 1)
        replica_restaurants_df.at[i, 'end_latitude'] = restaurant_sg_df.at[idxs,'latitude']
        replica_restaurants_df.at[i, 'end_longitude'] = restaurant_sg_df.at[idxs,'longitude']
replica_restaurants_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.62290058e+19,EAT,EAT,2023-02-03 00:34:13,2023-02-03 00:34:32,0,0.01652482,NaN,250173522001,42.3720955,...,mixed_use,retail,mixed_use,multi_family,20,22d-222@62j-sj2-vzz,42.371281,-71.083318,42.372122,-71.087128
1,8.46274435e+18,EAT,EAT,2023-02-03 00:47:19,2023-02-03 01:12:44,25,1.31656236,NaN,250173530002,42.3668330,...,commercial,retail,commercial,retail,20,zzw-224@62j-sjv-835,42.374306,-71.119126,42.369072,-71.100965
2,9.27219531e+18,EAT,EAT,2023-02-03 01:16:28,2023-02-03 01:16:28,0,0.00000000,NaN,250173540003,42.3766435,...,commercial,retail,commercial,retail,20,23k-223@62j-sjv-99f,42.373493,-71.120642,42.374228,-71.120007
3,2.21513782e+18,EAT,EAT,2023-02-03 05:52:44,2023-02-03 05:53:24,0,0.03484630,NaN,250173530002,42.3668330,...,commercial,retail,commercial,retail,20,zzw-225@62j-sj2-p35,42.365034,-71.102754,42.365034,-71.102754
4,5.38198951e+18,EAT,EAT,2023-02-03 06:05:35,2023-02-03 06:06:12,0,0.03244217,NaN,250173537004,42.3721570,...,commercial,retail,civic_institutional,education,20,22d-222@62j-sjv-8qf,42.372303,-71.116061,42.372479,-71.117929


In [226]:
# Save the relevant data for the GAMA simulation model
restaurant_trips_df=replica_restaurants_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                            'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
restaurant_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.62290058e+19,42.371281,-71.083318,42.372122,-71.087128,EAT,2023-02-03 00:34:13,2023-02-03 00:34:32,0,0.01652482,20
1,8.46274435e+18,42.374306,-71.119126,42.369072,-71.100965,EAT,2023-02-03 00:47:19,2023-02-03 01:12:44,25,1.31656236,20
2,9.27219531e+18,42.373493,-71.120642,42.374228,-71.120007,EAT,2023-02-03 01:16:28,2023-02-03 01:16:28,0,0.00000000,20
3,2.21513782e+18,42.365034,-71.102754,42.365034,-71.102754,EAT,2023-02-03 05:52:44,2023-02-03 05:53:24,0,0.03484630,20
4,5.38198951e+18,42.372303,-71.116061,42.372479,-71.117929,EAT,2023-02-03 06:05:35,2023-02-03 06:06:12,0,0.03244217,20


### WORK 

In [227]:
# Merge the WORK buildings: mixed use and commercial ones with origin land use 2 = retail with previous activity WORK
replica_work_df=pd.concat([retail_work_mu_df,retail_work_c_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_work_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,1.74210732e+19,EAT,WORK,2023-02-03 05:41:33,2023-02-03 05:41:34,0,0.00140397,NaN,250173531011,42.3616040,...,42.3616040,-71.09919947,commercial,retail,commercial,retail,20,25d-222@62j-sj2-p35,42.364343,-71.102333
1,1.28635372e+19,EAT,WORK,2023-02-03 05:48:03,2023-02-03 05:48:05,0,0.00187972,NaN,250173537004,42.3721570,...,42.3721570,-71.11732789,commercial,retail,commercial,retail,20,248-223@62j-sjv-b6k,42.372441,-71.118977
2,1.45201968e+19,EAT,WORK,2023-02-03 05:54:30,2023-02-03 05:58:54,4,0.22797026,NaN,250173531022,42.3585020,...,42.3585020,-71.08549670,commercial,retail,civic_institutional,education,20,zzw-223@62j-sj2-b49,42.359113,-71.092946
3,1.21960461e+19,EAT,MAINTENANCE,2023-02-03 06:18:49,2023-02-03 06:23:14,4,0.22888771,NaN,250173527001,42.3731030,...,42.3720955,-71.08541594,mixed_use,retail,mixed_use,retail,20,22d-222@62j-sj2-vzz,42.371281,-71.083318
4,2.38533435e+18,EAT,WORK,2023-02-03 06:22:13,2023-02-03 06:22:14,0,0.00168197,NaN,250173540003,42.3766435,...,42.3766435,-71.12102016,mixed_use,retail,mixed_use,retail,20,22h-222@62j-sjv-9mk,42.374228,-71.120007


In [228]:
# Concat the SHOPPING, HOTELS and RESTAURANTS shapefiles: because the people can work in any of these buildings
work_sg_df=pd.concat([shop_sg_df,hotel_sg_df,restaurant_sg_df]).reset_index().drop('index',axis=1)

# Based on the trip origin lat,lon assign the closest work building with function KDTree
pd.set_option("display.precision", 8)
replica_work_df['end_latitude']=''
replica_work_df['end_longitude']=''
for i in range(len(replica_work_df)):
    blockgroup = replica_work_df.at[i,'origin_bgrp']
    work_bgrp_sg_df = work_sg_df[work_sg_df['poi_cbg']==blockgroup]
    work_bgrp_sg_df = work_bgrp_sg_df.reset_index().drop('index',axis=1)
    if work_bgrp_sg_df.empty == False:
        placekey = work_bgrp_sg_df['placekey']
        placekey_selected = np.random.choice(placekey, 1)
        for j in range(len(work_bgrp_sg_df)):
            if work_bgrp_sg_df.at[j,'placekey'] == placekey_selected:
                replica_work_df.at[i, 'end_latitude'] = work_bgrp_sg_df.at[j,'latitude']
                replica_work_df.at[i, 'end_longitude'] = work_bgrp_sg_df.at[j,'longitude']
                break
    else:
        Z = np.array(work_sg_df[['latitude', 'longitude']])
        tree = spatial.KDTree(Z)
        dist, idxs = tree.query(np.array(replica_work_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 1)
        replica_work_df.at[i, 'end_latitude'] = work_sg_df.at[idxs,'latitude']
        replica_work_df.at[i, 'end_longitude'] = work_sg_df.at[idxs,'longitude']
replica_work_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.74210732e+19,EAT,WORK,2023-02-03 05:41:33,2023-02-03 05:41:34,0,0.00140397,NaN,250173531011,42.3616040,...,commercial,retail,commercial,retail,20,25d-222@62j-sj2-p35,42.364343,-71.102333,42.362335,-71.099042
1,1.28635372e+19,EAT,WORK,2023-02-03 05:48:03,2023-02-03 05:48:05,0,0.00187972,NaN,250173537004,42.3721570,...,commercial,retail,commercial,retail,20,248-223@62j-sjv-b6k,42.372441,-71.118977,42.372303,-71.116061
2,1.45201968e+19,EAT,WORK,2023-02-03 05:54:30,2023-02-03 05:58:54,4,0.22797026,NaN,250173531022,42.3585020,...,commercial,retail,civic_institutional,education,20,zzw-223@62j-sj2-b49,42.359113,-71.092946,42.359142,-71.092973
3,1.21960461e+19,EAT,MAINTENANCE,2023-02-03 06:18:49,2023-02-03 06:23:14,4,0.22888771,NaN,250173527001,42.3731030,...,mixed_use,retail,mixed_use,retail,20,22d-222@62j-sj2-vzz,42.371281,-71.083318,42.372614,-71.090856
4,2.38533435e+18,EAT,WORK,2023-02-03 06:22:13,2023-02-03 06:22:14,0,0.00168197,NaN,250173540003,42.3766435,...,mixed_use,retail,mixed_use,retail,20,22h-222@62j-sjv-9mk,42.374228,-71.120007,42.373551,-71.120249


In [229]:
# Save the relevant data for the GAMA simulation model
work_trips_df=replica_work_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                               'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
work_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.74210732e+19,42.364343,-71.102333,42.362335,-71.099042,EAT,2023-02-03 05:41:33,2023-02-03 05:41:34,0,0.00140397,20
1,1.28635372e+19,42.372441,-71.118977,42.372303,-71.116061,EAT,2023-02-03 05:48:03,2023-02-03 05:48:05,0,0.00187972,20
2,1.45201968e+19,42.359113,-71.092946,42.359142,-71.092973,EAT,2023-02-03 05:54:30,2023-02-03 05:58:54,4,0.22797026,20
3,1.21960461e+19,42.371281,-71.083318,42.372614,-71.090856,EAT,2023-02-03 06:18:49,2023-02-03 06:23:14,4,0.22888771,20
4,2.38533435e+18,42.374228,-71.120007,42.373551,-71.120249,EAT,2023-02-03 06:22:13,2023-02-03 06:22:14,0,0.00168197,20


## NON RETAIL ATTRACTION 

In [230]:
# Merge the NON RETAIL ATTRACTION use buildings: mixed use ones with origin land use 2 = non retail
replica_non_retail_df=pd.concat([replica_non_retail_mu_df,replica_non_retail_c_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_non_retail_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,1.42282106e+19,EAT,HOME,2023-02-03 04:14:49,2023-02-03 04:15:32,0,0.03732451,NaN,250173538003,42.371381,...,42.371381,-71.11125820,mixed_use,non_retail_attraction,mixed_use,non_retail_attraction,20,26n-225@62j-sjv-j7q,42.370698,-71.113341
1,4.76310803e+18,EAT,HOME,2023-02-03 05:38:49,2023-02-03 05:50:32,11,0.60707299,NaN,250173523001,42.367571,...,42.358502,-71.08549670,mixed_use,non_retail_attraction,civic_institutional,education,20,zzw-223@62j-sj2-b49,42.359113,-71.092946
2,5.74877875e+18,EAT,HOME,2023-02-03 05:38:49,2023-02-03 05:51:53,13,0.67506288,NaN,250173523001,42.367571,...,42.358502,-71.08549670,mixed_use,non_retail_attraction,civic_institutional,education,20,zzw-222@62j-sj4-rp9,42.359190,-71.088212
3,1.09018521e+19,EAT,HOME,2023-02-03 05:38:49,2023-02-03 05:41:06,2,1.09069964,NaN,250173523001,42.367571,...,42.358502,-71.08549670,mixed_use,non_retail_attraction,civic_institutional,education,20,222-222@62j-sj4-vcq,42.360213,-71.085994
4,5.89344909e+18,EAT,HOME,2023-02-03 06:37:37,2023-02-03 06:41:38,4,1.57624539,NaN,250173523001,42.367571,...,42.356422,-71.10057121,mixed_use,non_retail_attraction,civic_institutional,education,20,zzy-222@62j-sj2-ckf,42.359066,-71.094744


In [231]:
# SafeGraph Data for NON RETAIL ATTRACTION buildings
non_retail_poi_list=['Sporting Goods, Hobby, and Musical Instrument Stores','Museums, Historical Sites, and Similar Institutions',
                    'Amusement Parks and Arcades','Performing Arts Companies','Promoters of Performing Arts, Sports, and Similar Events',
                    'Religious Organizations']
non_retail_sg_df = safegraph_df[safegraph_df['top_category'].isin(non_retail_poi_list)]
non_retail_sg_df = non_retail_sg_df.reset_index().drop('index',axis=1)
non_retail_sg_df.head()

,placekey,latitude,longitude,top_category,poi_cbg
0,222-222@62j-sj2-sqz,42.365577,-71.105894,Religious Organizations,2.50173535e+11
1,zzz-222@62j-sj2-rzf,42.363428,-71.100044,"Museums, Historical Sites, and Similar Institu...",2.50173531e+11
2,223-224@62j-sjv-j5f,42.368903,-71.110873,"Museums, Historical Sites, and Similar Institu...",NaN
3,225-222@62j-sj2-sdv,42.363189,-71.098985,Religious Organizations,NaN
4,228-222@62j-sj3-9mk,42.373806,-71.100074,"Promoters of Performing Arts, Sports, and Simi...",2.50173527e+11


In [232]:
# Based on the trip origin lat,lon assign the closest NON RETAIL ATTRACTION building with function KDTree
pd.set_option("display.precision", 8)
replica_non_retail_df['end_latitude']=''
replica_non_retail_df['end_longitude']=''
for i in range(len(replica_non_retail_df)):
    blockgroup = replica_non_retail_df.at[i,'origin_bgrp']
    non_retail_bgrp_sg_df = non_retail_sg_df[non_retail_sg_df['poi_cbg']==blockgroup]
    non_retail_bgrp_sg_df = non_retail_bgrp_sg_df.reset_index().drop('index',axis=1)
    if non_retail_bgrp_sg_df.empty == False:
        placekey = non_retail_bgrp_sg_df['placekey']
        placekey_selected = np.random.choice(placekey, 1)
        for j in range(len(non_retail_bgrp_sg_df)):
            if non_retail_bgrp_sg_df.at[j,'placekey'] == placekey_selected:
                replica_non_retail_df.at[i, 'end_latitude'] = non_retail_bgrp_sg_df.at[j,'latitude']
                replica_non_retail_df.at[i, 'end_longitude'] = non_retail_bgrp_sg_df.at[j,'longitude']
                break
    else:
        Z = np.array(non_retail_sg_df[['latitude', 'longitude']])
        tree = spatial.KDTree(Z)
        dist, idxs = tree.query(np.array(replica_non_retail_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 1)
        replica_non_retail_df.at[i, 'end_latitude'] = non_retail_sg_df.at[idxs,'latitude']
        replica_non_retail_df.at[i, 'end_longitude'] = non_retail_sg_df.at[idxs,'longitude']
replica_non_retail_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.42282106e+19,EAT,HOME,2023-02-03 04:14:49,2023-02-03 04:15:32,0,0.03732451,NaN,250173538003,42.371381,...,mixed_use,non_retail_attraction,mixed_use,non_retail_attraction,20,26n-225@62j-sjv-j7q,42.370698,-71.113341,42.372684,-71.110008
1,4.76310803e+18,EAT,HOME,2023-02-03 05:38:49,2023-02-03 05:50:32,11,0.60707299,NaN,250173523001,42.367571,...,mixed_use,non_retail_attraction,civic_institutional,education,20,zzw-223@62j-sj2-b49,42.359113,-71.092946,42.370543,-71.085226
2,5.74877875e+18,EAT,HOME,2023-02-03 05:38:49,2023-02-03 05:51:53,13,0.67506288,NaN,250173523001,42.367571,...,mixed_use,non_retail_attraction,civic_institutional,education,20,zzw-222@62j-sj4-rp9,42.359190,-71.088212,42.370543,-71.085226
3,1.09018521e+19,EAT,HOME,2023-02-03 05:38:49,2023-02-03 05:41:06,2,1.09069964,NaN,250173523001,42.367571,...,mixed_use,non_retail_attraction,civic_institutional,education,20,222-222@62j-sj4-vcq,42.360213,-71.085994,42.370543,-71.085226
4,5.89344909e+18,EAT,HOME,2023-02-03 06:37:37,2023-02-03 06:41:38,4,1.57624539,NaN,250173523001,42.367571,...,mixed_use,non_retail_attraction,civic_institutional,education,20,zzy-222@62j-sj2-ckf,42.359066,-71.094744,42.370543,-71.085226


In [233]:
# Save the relevant data for the GAMA simulation model
non_retail_trips_df=replica_non_retail_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                         'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
non_retail_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.42282106e+19,42.370698,-71.113341,42.372684,-71.110008,EAT,2023-02-03 04:14:49,2023-02-03 04:15:32,0,0.03732451,20
1,4.76310803e+18,42.359113,-71.092946,42.370543,-71.085226,EAT,2023-02-03 05:38:49,2023-02-03 05:50:32,11,0.60707299,20
2,5.74877875e+18,42.359190,-71.088212,42.370543,-71.085226,EAT,2023-02-03 05:38:49,2023-02-03 05:51:53,13,0.67506288,20
3,1.09018521e+19,42.360213,-71.085994,42.370543,-71.085226,EAT,2023-02-03 05:38:49,2023-02-03 05:41:06,2,1.09069964,20
4,5.89344909e+18,42.359066,-71.094744,42.370543,-71.085226,EAT,2023-02-03 06:37:37,2023-02-03 06:41:38,4,1.57624539,20


## PARK 

In [234]:
# Filter all the OPEN SPACE use buildings (based on origin land use 1)
replica_park_df = eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='open_space'].reset_index().drop('index',axis=1)

In [235]:
# Drop OPEN SPACE use trips to check if all of them are assigned at the end
drop_list = ['open_space']
eattrips_df_copy = eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False]

In [236]:
# Merge de OPEN SPACE use buildings with the mixed use and commercial ones with origin land use 2 = open_space and  origin land use 2 = retail with previous activity RECREATION
replica_park_df=pd.concat([replica_park_df,retail_park_c_df,replica_park_mu_df,retail_park_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_park_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,5.09076771e+18,EAT,RECREATION,2023-02-03 05:50:11,2023-02-03 05:53:34,3,0.17562001,NaN,250173531012,42.363357,...,42.3633570,-71.09849179,open_space,open_space,civic_institutional,education,20,239-222@62j-sj2-rzf,42.363652,-71.099990
1,1.29438605e+18,EAT,RECREATION,2023-02-03 06:35:05,2023-02-03 06:37:13,2,0.87914011,NaN,250173537002,42.375375,...,42.3662725,-71.10594473,open_space,open_space,commercial,retail,20,226-222@62j-sj2-t35,42.365967,-71.105213
2,8.45078222e+18,EAT,RECREATION,2023-02-03 06:56:33,2023-02-03 06:58:18,1,0.64447358,NaN,250173537002,42.375375,...,42.3740220,-71.09943520,open_space,open_space,mixed_use,multi_family,20,22c-222@62j-sj3-9mk,42.373813,-71.100153
3,8.86366673e+18,EAT,RECREATION,2023-02-03 07:16:29,2023-02-03 07:17:10,0,0.03554929,NaN,250173530002,42.366833,...,42.3668330,-71.10146531,commercial,retail,commercial,retail,20,23p-222@62j-sj2-5j9,42.369072,-71.100965
4,7.92169605e+18,EAT,SCHOOL,2023-02-03 08:22:40,2023-02-03 08:26:16,3,1.11282829,NaN,250173537002,42.375375,...,42.3721570,-71.11732789,open_space,open_space,commercial,retail,20,23y-222@62j-sjv-bc5,42.372585,-71.119869


In [237]:
# Load the PARK shapefile data
filepath_park_csv = ".././02 Data/Shapefile/Tangible-Table/"
park_df = pd.read_csv(filepath_residential_csv + 'tangibletable_parkbuildings.csv')
park_df.head()

,Name,Shape_Leng,Shape_Area,lon_POI,lat_POI,usage,geometry
0,1407 CAMBRIDGE ST,327.22184105,6039.15356445,-71.10148902,42.37399274,P,POLYGON ((-71.10167552568294 42.37392953450808...
1,971 MEMORIAL DR,3007.31745017,220918.86987300,-71.12310163,42.37091709,P,POLYGON ((-71.12202422346149 42.37222278049008...
2,65 SIDNEY ST,1205.72965880,54051.00146480,-71.10158191,42.36150627,P,"POLYGON ((-71.1019387387137 42.3609214534923, ..."
3,83 WINTHROP ST,431.57500197,10994.55029300,-71.12055953,42.37244485,P,POLYGON ((-71.12082190390532 42.37234573668075...
4,1573 MASSACHUSETTS AVE,814.78267350,12979.57055660,-71.12037217,42.37827013,P,POLYGON ((-71.12034454148288 42.37785219931816...


In [238]:
# Based on the trip origin lat,lon assign the closest PARK building with function KDTree
pd.set_option("display.precision", 8)
replica_park_df['end_latitude']=''
replica_park_df['end_longitude']=''
Z = np.array(park_df[['lat_POI', 'lon_POI']])
tree = spatial.KDTree(Z)
for i in range(len(replica_park_df)):
    dist, idxs = tree.query(np.array(replica_park_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 5)
    dist_selected = np.random.choice(dist, 1)
    for j in range(5):
        if dist[j] == dist_selected:
            idx_selected = idxs[j]
            break
    replica_park_df.at[i, 'end_latitude'] = park_df.at[idx_selected,'lat_POI']
    replica_park_df.at[i, 'end_longitude'] = park_df.at[idx_selected,'lon_POI']
replica_park_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,5.09076771e+18,EAT,RECREATION,2023-02-03 05:50:11,2023-02-03 05:53:34,3,0.17562001,NaN,250173531012,42.363357,...,open_space,open_space,civic_institutional,education,20,239-222@62j-sj2-rzf,42.363652,-71.099990,42.362849,-71.099956
1,1.29438605e+18,EAT,RECREATION,2023-02-03 06:35:05,2023-02-03 06:37:13,2,0.87914011,NaN,250173537002,42.375375,...,open_space,open_space,commercial,retail,20,226-222@62j-sj2-t35,42.365967,-71.105213,42.369524,-71.10782
2,8.45078222e+18,EAT,RECREATION,2023-02-03 06:56:33,2023-02-03 06:58:18,1,0.64447358,NaN,250173537002,42.375375,...,open_space,open_space,mixed_use,multi_family,20,22c-222@62j-sj3-9mk,42.373813,-71.100153,42.373336,-71.104235
3,8.86366673e+18,EAT,RECREATION,2023-02-03 07:16:29,2023-02-03 07:17:10,0,0.03554929,NaN,250173530002,42.366833,...,commercial,retail,commercial,retail,20,23p-222@62j-sj2-5j9,42.369072,-71.100965,42.371006,-71.101488
4,7.92169605e+18,EAT,SCHOOL,2023-02-03 08:22:40,2023-02-03 08:26:16,3,1.11282829,NaN,250173537002,42.375375,...,open_space,open_space,commercial,retail,20,23y-222@62j-sjv-bc5,42.372585,-71.119869,42.373336,-71.104235


In [239]:
# Save the relevant data for the GAMA simulation model
park_trips_df=replica_park_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                               'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
park_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,5.09076771e+18,42.363652,-71.099990,42.362849,-71.099956,EAT,2023-02-03 05:50:11,2023-02-03 05:53:34,3,0.17562001,20
1,1.29438605e+18,42.365967,-71.105213,42.369524,-71.10782,EAT,2023-02-03 06:35:05,2023-02-03 06:37:13,2,0.87914011,20
2,8.45078222e+18,42.373813,-71.100153,42.373336,-71.104235,EAT,2023-02-03 06:56:33,2023-02-03 06:58:18,1,0.64447358,20
3,8.86366673e+18,42.369072,-71.100965,42.371006,-71.101488,EAT,2023-02-03 07:16:29,2023-02-03 07:17:10,0,0.03554929,20
4,7.92169605e+18,42.372585,-71.119869,42.373336,-71.104235,EAT,2023-02-03 08:22:40,2023-02-03 08:26:16,3,1.11282829,20


## TRANSPORTATION UTILITIES 

In [240]:
# Filter all the TRANSPORTATION UTILITIES use buildings (based on origin land use 1)
replica_transportationu_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='transportation_utilities'].reset_index().drop('index',axis=1)

In [241]:
# Drop TRANSPORTATION UTILITIES use trips to check if all of them are assigned at the end
drop_list=['transportation_utilities']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False]

In [242]:
# Merge de TRANSPORTATION UTILITIES use buildings with the mixed use and commercial ones with origin land use 2 = open_space and  origin land use 2 = retail with previous activity RECREATION
replica_transportationu_df=pd.concat([replica_transportationu_df,replica_tu_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_transportationu_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,6.92349052e+18,EAT,WORK,2023-02-03 07:54:27,2023-02-03 07:59:12,4,1.69074424,NaN,250173540003,42.3766435,...,42.3768545,-71.09241285,mixed_use,transportation_utilities,commercial,retail,20,225-222@62j-sj2-z2k,42.377959,-71.089727
1,8.53350404e+18,EAT,WORK,2023-02-03 08:52:57,2023-02-03 09:00:52,7,0.40999907,NaN,250173530001,42.3662725,...,42.3713810,-71.11125820,mixed_use,transportation_utilities,commercial,retail,20,zzy-226@62j-sj2-jn5,42.369345,-71.110752
2,6.56013819e+18,EAT,WORK,2023-02-03 10:51:12,2023-02-03 10:54:42,3,0.18144600,NaN,250173529002,42.3734860,...,42.3740220,-71.09943520,mixed_use,transportation_utilities,commercial,retail,20,22v-224@62j-sj3-9cq,42.374052,-71.100069
3,1.36145279e+19,EAT,SHOP,2023-02-03 11:01:04,2023-02-03 11:08:23,7,1.86526193,NaN,250173540003,42.3766435,...,42.3668330,-71.10146531,mixed_use,transportation_utilities,commercial,retail,20,22b-222@62j-sj2-rff,42.365028,-71.102423
4,6.52516755e+18,EAT,OTHER_ACTIVITY_TYPE,2023-02-03 11:27:34,2023-02-03 11:29:47,2,0.53159759,NaN,250173527002,42.3736705,...,42.3768545,-71.09241285,mixed_use,transportation_utilities,commercial,non_retail_attraction,20,237-222@62j-sj3-b8v,42.379450,-71.096153


In [243]:
# Load the transportation OSM data
list_osm_transportation = ['bicycle_parking','bicycle_repair_station','bicycle_rental','boat_rental','boat_sharing','bus_station',
                         'car_rental','car_sharing','car_wash','vehicle_inspection','charging_station','ferry_terminal','fuel',
                         'grit_bin','motorcycle_parking','parking','parking_entrance','parking_space','taxi']
osm_transportation_df = osm_amenities_df[osm_amenities_df['amenity'].isin(list_osm_transportation)].reset_index().drop('index',axis=1)
osm_transportation_df.head()

,id,amenity,name,lat,lon
0,69504598,bus_station,Harvard Square,42.3733695,-71.1189903
1,441287631,bicycle_parking,LiveableStreets,42.3598870,-71.1029331
2,505869274,parking_entrance,NaN,42.3739663,-71.1113252
3,505869282,parking,NaN,42.3737921,-71.1112861
4,559506931,car_sharing,NaN,42.3800832,-71.0955128


In [244]:
# Based on the trip origin lat,lon assign the closest TRANSPORTATION UTILITY with function KDTree
pd.set_option("display.precision", 10)
replica_transportationu_df['end_latitude']=''
replica_transportationu_df['end_longitude']=''
Z = np.array(osm_transportation_df[['lat', 'lon']])
tree = spatial.KDTree(Z)
for i in range(len(replica_transportationu_df)):
    dist, idxs = tree.query(np.array(replica_transportationu_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 5)
    dist_selected = np.random.choice(dist, 1)
    for j in range(5):
        if dist[j] == dist_selected:
            idx_selected = idxs[j]
            break
    replica_transportationu_df.at[i, 'end_latitude'] = osm_transportation_df.at[idx_selected,'lat']
    replica_transportationu_df.at[i, 'end_longitude'] = osm_transportation_df.at[idx_selected,'lon']
replica_transportationu_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,6.9234905221e+18,EAT,WORK,2023-02-03 07:54:27,2023-02-03 07:59:12,4,1.6907442406,NaN,250173540003,42.3766435,...,mixed_use,transportation_utilities,commercial,retail,20,225-222@62j-sj2-z2k,42.377959,-71.089727,42.3774932,-71.1218498
1,8.5335040444e+18,EAT,WORK,2023-02-03 08:52:57,2023-02-03 09:00:52,7,0.4099990668,NaN,250173530001,42.3662725,...,mixed_use,transportation_utilities,commercial,retail,20,zzy-226@62j-sj2-jn5,42.369345,-71.110752,42.3668757,-71.1063739
2,6.5601381926e+18,EAT,WORK,2023-02-03 10:51:12,2023-02-03 10:54:42,3,0.1814460011,NaN,250173529002,42.3734860,...,mixed_use,transportation_utilities,commercial,retail,20,22v-224@62j-sj3-9cq,42.374052,-71.100069,42.3739463,-71.1029765
3,1.3614527932e+19,EAT,SHOP,2023-02-03 11:01:04,2023-02-03 11:08:23,7,1.8652619276,NaN,250173540003,42.3766435,...,mixed_use,transportation_utilities,commercial,retail,20,22b-222@62j-sj2-rff,42.365028,-71.102423,42.377615,-71.122186
4,6.5251675476e+18,EAT,OTHER_ACTIVITY_TYPE,2023-02-03 11:27:34,2023-02-03 11:29:47,2,0.5315975880,NaN,250173527002,42.3736705,...,mixed_use,transportation_utilities,commercial,non_retail_attraction,20,237-222@62j-sj3-b8v,42.379450,-71.096153,42.3730344,-71.0943444


In [245]:
# Save the relevant data for the GAMA simulation model
transportationutilities_trips_df=replica_transportationu_df[['activity_id','start_latitude','start_longitude','end_latitude',
                                                             'end_longitude','travel_purpose','start_time','end_time',
                                                             'duration_minutes','distance_miles','start_hour']]
transportationutilities_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,6.9234905221e+18,42.377959,-71.089727,42.3774932,-71.1218498,EAT,2023-02-03 07:54:27,2023-02-03 07:59:12,4,1.6907442406,20
1,8.5335040444e+18,42.369345,-71.110752,42.3668757,-71.1063739,EAT,2023-02-03 08:52:57,2023-02-03 09:00:52,7,0.4099990668,20
2,6.5601381926e+18,42.374052,-71.100069,42.3739463,-71.1029765,EAT,2023-02-03 10:51:12,2023-02-03 10:54:42,3,0.1814460011,20
3,1.3614527932e+19,42.365028,-71.102423,42.377615,-71.122186,EAT,2023-02-03 11:01:04,2023-02-03 11:08:23,7,1.8652619276,20
4,6.5251675476e+18,42.379450,-71.096153,42.3730344,-71.0943444,EAT,2023-02-03 11:27:34,2023-02-03 11:29:47,2,0.5315975880,20


## CIVIC INSTITUTIONAL 

In [246]:
# Merge de CIVIC INSTITUTIONAL use buildings: mixed use and civic institutional ones with origin land use 2 = civic institutional
replica_civic_institutional_df=pd.concat([replica_civic_institutional_ci_df,replica_civic_institutional_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_civic_institutional_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,8.7250591691e+18,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:02:13,2,1.2559789496,NaN,250173537005,42.3739040,...,42.3781865,-71.10417041,civic_institutional,civic_institutional,commercial,retail,20,237-222@62j-sk2-pd9,42.377298,-71.104867
1,1.0448849400e+19,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:20:18,21,1.0904240197,NaN,250173537005,42.3739040,...,42.3781865,-71.10417041,civic_institutional,civic_institutional,commercial,retail,20,237-222@62j-sk2-pd9,42.377298,-71.104867
2,1.4539978825e+19,EAT,MAINTENANCE,2023-02-03 03:16:46,2023-02-03 03:39:49,23,1.1936032601,NaN,250173537001,42.3750675,...,42.3616040,-71.09919947,mixed_use,civic_institutional,commercial,non_retail_attraction,20,25d-222@62j-sj2-p35,42.364343,-71.102333
3,1.1814714570e+19,EAT,MAINTENANCE,2023-02-03 04:47:23,2023-02-03 04:49:24,2,0.1044953286,NaN,250173515002,42.3768545,...,42.3768545,-71.09241285,civic_institutional,civic_institutional,commercial,non_retail_attraction,20,225-222@62j-sj2-z2k,42.377959,-71.089727
4,8.4695038543e+18,EAT,HOME,2023-02-03 04:49:38,2023-02-03 04:52:06,2,0.1279504078,NaN,250173539001,42.3696330,...,42.3713810,-71.11125820,civic_institutional,civic_institutional,mixed_use,non_retail_attraction,20,25w-226@62j-sjv-j35,42.369628,-71.111400


In [247]:
# Load the civic institutional OSM data
list_osm_civicinstitutional = ['courthouse','embassy','fire_station','police','post_box','post_depot','post_office','prison',
                             'ranger_station','townhall']
osm_civicinstitutional_df = osm_amenities_df[osm_amenities_df['amenity'].isin(list_osm_civicinstitutional)].reset_index().drop('index',axis=1)
osm_civicinstitutional_df.head()

,id,amenity,name,lat,lon
0,257493789,police,Cambridge Police Dept,42.3749188,-71.1144991
1,257493900,police,Cambridge Police Department,42.3672754,-71.0861518
2,257493974,police,Harvard University Police,42.3787447,-71.1237790
3,257494617,fire_station,Somerville Fire Department,42.3789022,-71.0925588
4,345825149,post_box,United States Postal Service,42.3793849,-71.1009090


In [248]:
# Based on the trip origin lat,lon assign the closest civic institutional buildings with function KDTree
pd.set_option("display.precision", 10)
replica_civic_institutional_df['end_latitude']=''
replica_civic_institutional_df['end_longitude']=''
Z = np.array(osm_civicinstitutional_df[['lat', 'lon']])
tree = spatial.KDTree(Z)
for i in range(len(replica_civic_institutional_df)):
    dist, idxs = tree.query(np.array(replica_civic_institutional_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 5)
    dist_selected = np.random.choice(dist, 1)
    for j in range(5):
        if dist[j] == dist_selected:
            idx_selected = idxs[j]
            break
    replica_civic_institutional_df.at[i, 'end_latitude'] = osm_civicinstitutional_df.at[idx_selected,'lat']
    replica_civic_institutional_df.at[i, 'end_longitude'] = osm_civicinstitutional_df.at[idx_selected,'lon']
replica_civic_institutional_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,8.7250591691e+18,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:02:13,2,1.2559789496,NaN,250173537005,42.3739040,...,civic_institutional,civic_institutional,commercial,retail,20,237-222@62j-sk2-pd9,42.377298,-71.104867,42.3726631,-71.1096359
1,1.0448849400e+19,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:20:18,21,1.0904240197,NaN,250173537005,42.3739040,...,civic_institutional,civic_institutional,commercial,retail,20,237-222@62j-sk2-pd9,42.377298,-71.104867,42.3699518,-71.1131418
2,1.4539978825e+19,EAT,MAINTENANCE,2023-02-03 03:16:46,2023-02-03 03:39:49,23,1.1936032601,NaN,250173537001,42.3750675,...,mixed_use,civic_institutional,commercial,non_retail_attraction,20,25d-222@62j-sj2-p35,42.364343,-71.102333,42.3748307,-71.1089476
3,1.1814714570e+19,EAT,MAINTENANCE,2023-02-03 04:47:23,2023-02-03 04:49:24,2,0.1044953286,NaN,250173515002,42.3768545,...,civic_institutional,civic_institutional,commercial,non_retail_attraction,20,225-222@62j-sj2-z2k,42.377959,-71.089727,42.3759383,-71.0974585
4,8.4695038543e+18,EAT,HOME,2023-02-03 04:49:38,2023-02-03 04:52:06,2,0.1279504078,NaN,250173539001,42.3696330,...,civic_institutional,civic_institutional,mixed_use,non_retail_attraction,20,25w-226@62j-sjv-j35,42.369628,-71.111400,42.3699518,-71.1131418


In [249]:
# Save the relevant data for the GAMA simulation model
civic_institutional_trips_df=replica_civic_institutional_df[['activity_id','start_latitude','start_longitude','end_latitude',
                                                           'end_longitude','travel_purpose','start_time','end_time',
                                                           'duration_minutes','distance_miles','start_hour']]
civic_institutional_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,8.7250591691e+18,42.377298,-71.104867,42.3726631,-71.1096359,EAT,2023-02-03 00:59:15,2023-02-03 01:02:13,2,1.2559789496,20
1,1.0448849400e+19,42.377298,-71.104867,42.3699518,-71.1131418,EAT,2023-02-03 00:59:15,2023-02-03 01:20:18,21,1.0904240197,20
2,1.4539978825e+19,42.364343,-71.102333,42.3748307,-71.1089476,EAT,2023-02-03 03:16:46,2023-02-03 03:39:49,23,1.1936032601,20
3,1.1814714570e+19,42.377959,-71.089727,42.3759383,-71.0974585,EAT,2023-02-03 04:47:23,2023-02-03 04:49:24,2,0.1044953286,20
4,8.4695038543e+18,42.369628,-71.111400,42.3699518,-71.1131418,EAT,2023-02-03 04:49:38,2023-02-03 04:52:06,2,0.1279504078,20


### EDUCATION 

In [250]:
# Merge de EDUCATION use buildings: mixed use, civic institutional, commercial ones with origin land use 2 = education or  with origin land use 2 = retail and previous activity type = SCHOOL
replica_education_df = pd.concat([replica_education_ci_df,retail_education_c_df,replica_education_mu_df,retail_education_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_education_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,1.2174187897e+19,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:03:16,4,1.7446100187,NaN,250173539003,42.3709185,...,42.3781865,-71.10417041,civic_institutional,education,mixed_use,multi_family,20,237-222@62j-sk2-pd9,42.377298,-71.104867
1,1.7766032573e+19,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:22:06,22,1.1836526323,NaN,250173539003,42.3709185,...,42.3781865,-71.10417041,civic_institutional,education,mixed_use,multi_family,20,22g-222@62j-sk2-pjv,42.378182,-71.106010
2,4.3909919005e+18,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:02:28,3,1.3801832846,NaN,250173539001,42.3696330,...,42.3781865,-71.10417041,civic_institutional,education,commercial,retail,20,237-222@62j-sk2-pd9,42.377298,-71.104867
3,1.5976922764e+19,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:00:58,1,0.7394306762,NaN,250173537003,42.3743180,...,42.3721570,-71.11732789,civic_institutional,education,civic_institutional,education,20,227-222@62j-sjv-8qf,42.372208,-71.115966
4,1.6915641606e+19,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:01:21,2,0.6905597540,NaN,250173539003,42.3709185,...,42.3709185,-71.11886070,civic_institutional,education,commercial,retail,20,223-222@62j-sjv-9xq,42.370282,-71.118519


In [251]:
# SafeGraph Data for EDUCATION buildings
education_poi_list = ['Other Schools and Instruction','Colleges, Universities, and Professional Schools','Technical and Trade Schools',
                    'Administration of Economic Programs','Elementary and Secondary Schools','Child Day Care Services']
education_sg_df = safegraph_df[safegraph_df['top_category'].isin(education_poi_list)]
education_sg_df = education_sg_df.reset_index().drop('index',axis=1)
education_sg_df.head()

,placekey,latitude,longitude,top_category,poi_cbg
0,22f-222@62j-sj2-p35,42.364208,-71.102297,Other Schools and Instruction,NaN
1,223-222@62j-sj3-4y9,42.369468,-71.083053,Child Day Care Services,250173521021.0
2,226-222@62j-sj3-btv,42.377576,-71.093600,Child Day Care Services,NaN
3,22t-223@62j-sjv-r49,42.378529,-71.123497,"Colleges, Universities, and Professional Schools",250173540003.0
4,226-222@62j-sjv-b8v,42.371219,-71.117438,"Colleges, Universities, and Professional Schools",250173539003.0


In [252]:
# Based on the trip origin lat,lon assign the closest EDUCATION building with function KDTree
pd.set_option("display.precision", 8)
replica_education_df['end_latitude']=''
replica_education_df['end_longitude']=''
for i in range(len(replica_education_df)):
    blockgroup = replica_education_df.at[i,'origin_bgrp']
    education_bgrp_sg_df = education_sg_df[education_sg_df['poi_cbg']==blockgroup]
    education_bgrp_sg_df = education_bgrp_sg_df.reset_index().drop('index',axis=1)
    if education_bgrp_sg_df.empty == False:
        placekey = education_bgrp_sg_df['placekey']
        placekey_selected = np.random.choice(placekey, 1)
        for j in range(len(education_bgrp_sg_df)):
            if education_bgrp_sg_df.at[j,'placekey'] == placekey_selected:
                replica_education_df.at[i, 'end_latitude'] = education_bgrp_sg_df.at[j,'latitude']
                replica_education_df.at[i, 'end_longitude'] = education_bgrp_sg_df.at[j,'longitude']
                break
    else:
        Z = np.array(education_sg_df[['latitude', 'longitude']])
        tree = spatial.KDTree(Z)
        dist, idxs = tree.query(np.array(replica_education_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 1)
        replica_education_df.at[i, 'end_latitude'] = education_sg_df.at[idxs,'latitude']
        replica_education_df.at[i, 'end_longitude'] = education_sg_df.at[idxs,'longitude']
replica_education_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.21741879e+19,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:03:16,4,1.74461002,NaN,250173539003,42.3709185,...,civic_institutional,education,mixed_use,multi_family,20,237-222@62j-sk2-pd9,42.377298,-71.104867,42.372042,-71.119947
1,1.77660326e+19,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:22:06,22,1.18365263,NaN,250173539003,42.3709185,...,civic_institutional,education,mixed_use,multi_family,20,22g-222@62j-sk2-pjv,42.378182,-71.106010,42.371219,-71.117438
2,4.39099190e+18,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:02:28,3,1.38018328,NaN,250173539001,42.3696330,...,civic_institutional,education,commercial,retail,20,237-222@62j-sk2-pd9,42.377298,-71.104867,42.370917,-71.114905
3,1.59769228e+19,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:00:58,1,0.73943068,NaN,250173537003,42.3743180,...,civic_institutional,education,civic_institutional,education,20,227-222@62j-sjv-8qf,42.372208,-71.115966,42.374182,-71.116059
4,1.69156416e+19,EAT,HOME,2023-02-03 00:59:15,2023-02-03 01:01:21,2,0.69055975,NaN,250173539003,42.3709185,...,civic_institutional,education,commercial,retail,20,223-222@62j-sjv-9xq,42.370282,-71.118519,42.372042,-71.119947


In [253]:
# Save the relevant data for the GAMA simulation model
education_trips_df=replica_education_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                         'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
education_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.21741879e+19,42.377298,-71.104867,42.372042,-71.119947,EAT,2023-02-03 00:59:15,2023-02-03 01:03:16,4,1.74461002,20
1,1.77660326e+19,42.378182,-71.106010,42.371219,-71.117438,EAT,2023-02-03 00:59:15,2023-02-03 01:22:06,22,1.18365263,20
2,4.39099190e+18,42.377298,-71.104867,42.370917,-71.114905,EAT,2023-02-03 00:59:15,2023-02-03 01:02:28,3,1.38018328,20
3,1.59769228e+19,42.372208,-71.115966,42.374182,-71.116059,EAT,2023-02-03 00:59:15,2023-02-03 01:00:58,1,0.73943068,20
4,1.69156416e+19,42.370282,-71.118519,42.372042,-71.119947,EAT,2023-02-03 00:59:15,2023-02-03 01:01:21,2,0.69055975,20


## HEALTHCARE 

In [254]:
# Merge de HEALTHCARE use buildings: mixed use, civic institutional ones with origin land use 2 = healthcare
replica_healthcare_df=pd.concat([replica_healthcare_ci_df,replica_healthcare_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_healthcare_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude
0,9.47840871e+17,EAT,MAINTENANCE,2023-02-03 10:19:03,2023-02-03 10:24:11,5,1.48284668,NaN,250173537001,42.3750675,...,42.3696330,-71.11517733,civic_institutional,healthcare,civic_institutional,education,20,26w-222@62j-sjv-kfz,42.368397,-71.116023
1,1.25025580e+19,EAT,MAINTENANCE,2023-02-03 10:28:52,2023-02-03 10:30:47,1,0.56743335,NaN,250173537001,42.3750675,...,42.3715445,-71.10118533,civic_institutional,healthcare,commercial,retail,20,222-222@62j-sj3-9fz,42.372746,-71.099375
2,7.85785543e+18,EAT,MAINTENANCE,2023-02-03 11:17:24,2023-02-03 11:19:51,2,0.57928027,NaN,250173537001,42.3750675,...,42.3740220,-71.09943520,civic_institutional,healthcare,mixed_use,retail,20,22v-224@62j-sj3-9cq,42.374052,-71.100069
3,1.37671630e+19,EAT,MAINTENANCE,2023-02-03 12:12:08,2023-02-03 12:15:18,3,0.87821869,NaN,250173537001,42.3750675,...,42.3721570,-71.11732789,civic_institutional,healthcare,commercial,retail,20,23f-222@62j-sjv-b6k,42.372895,-71.117919
4,1.46072351e+19,EAT,MAINTENANCE,2023-02-03 13:09:58,2023-02-03 13:29:30,19,1.01223210,NaN,250173537001,42.3750675,...,42.3768545,-71.09241285,civic_institutional,healthcare,commercial,retail,20,237-222@62j-sj3-b8v,42.379450,-71.096153


In [255]:
# SafeGraph Data for HEALTHCARE buildings
healthcare_poi_list = ['Offices of Other Health Practitioners','Offices of Physicians','Personal Care Services','Individual and Family Services',
                    'Outpatient Care Centers','Other Personal Services','Offices of Dentists','Home Health Care Services',
                    'Nursing Care Facilities (Skilled Nursing Facilities)','Medical and Diagnostic Laboratories','General Medical and Surgical Hospitals',
                    'Nursing and Residential Care Facilities','Death Care Services','Specialty (except Psychiatric and Substance Abuse) Hospitals',
                    'Insurance Carriers']
healthcare_sg_df = safegraph_df[safegraph_df['top_category'].isin(healthcare_poi_list)]
healthcare_sg_df = healthcare_sg_df.reset_index().drop('index',axis=1)
healthcare_sg_df.head()

,placekey,latitude,longitude,top_category,poi_cbg
0,zzw-224@62j-sj2-vcq,42.374391,-71.084562,Medical and Diagnostic Laboratories,NaN
1,zzy-223@62j-sj2-f2k,42.362251,-71.088964,Specialty (except Psychiatric and Substance Ab...,2.50173531e+11
2,25d-222@62j-sjv-j7q,42.370715,-71.113316,Offices of Other Health Practitioners,NaN
3,zzy-222@62j-sjv-j35,42.369430,-71.110975,Personal Care Services,NaN
4,22s-224@62j-sj2-rhq,42.366319,-71.103475,Personal Care Services,NaN


In [256]:
# Based on the trip origin lat,lon assign the closest HEALTHCARE building with function KDTree
pd.set_option("display.precision", 8)
replica_healthcare_df['end_latitude']=''
replica_healthcare_df['end_longitude']=''
for i in range(len(replica_healthcare_df)):
    blockgroup = replica_healthcare_df.at[i,'origin_bgrp']
    healthcare_bgrp_sg_df = healthcare_sg_df[healthcare_sg_df['poi_cbg']==blockgroup]
    healthcare_bgrp_sg_df = healthcare_bgrp_sg_df.reset_index().drop('index',axis=1)
    if healthcare_bgrp_sg_df.empty == False:
        placekey = healthcare_bgrp_sg_df['placekey']
        placekey_selected = np.random.choice(placekey, 1)
        for j in range(len(healthcare_bgrp_sg_df)):
            if healthcare_bgrp_sg_df.at[j,'placekey'] == placekey_selected:
                replica_healthcare_df.at[i, 'end_latitude'] = healthcare_bgrp_sg_df.at[j,'latitude']
                replica_healthcare_df.at[i, 'end_longitude'] = healthcare_bgrp_sg_df.at[j,'longitude']
                break
    else:
        Z = np.array(healthcare_sg_df[['latitude', 'longitude']])
        tree = spatial.KDTree(Z)
        dist, idxs = tree.query(np.array(replica_healthcare_df[['origin_bgrp_lat', 'origin_bgrp_lng']].iloc[i]), 1)
        replica_healthcare_df.at[i, 'end_latitude'] = healthcare_sg_df.at[idxs,'latitude']
        replica_healthcare_df.at[i, 'end_longitude'] = healthcare_sg_df.at[idxs,'longitude']
replica_healthcare_df.head()

,activity_id,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,...,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,9.47840871e+17,EAT,MAINTENANCE,2023-02-03 10:19:03,2023-02-03 10:24:11,5,1.48284668,NaN,250173537001,42.3750675,...,civic_institutional,healthcare,civic_institutional,education,20,26w-222@62j-sjv-kfz,42.368397,-71.116023,42.374964,-71.108469
1,1.25025580e+19,EAT,MAINTENANCE,2023-02-03 10:28:52,2023-02-03 10:30:47,1,0.56743335,NaN,250173537001,42.3750675,...,civic_institutional,healthcare,commercial,retail,20,222-222@62j-sj3-9fz,42.372746,-71.099375,42.375417,-71.10716
2,7.85785543e+18,EAT,MAINTENANCE,2023-02-03 11:17:24,2023-02-03 11:19:51,2,0.57928027,NaN,250173537001,42.3750675,...,civic_institutional,healthcare,mixed_use,retail,20,22v-224@62j-sj3-9cq,42.374052,-71.100069,42.374964,-71.108469
3,1.37671630e+19,EAT,MAINTENANCE,2023-02-03 12:12:08,2023-02-03 12:15:18,3,0.87821869,NaN,250173537001,42.3750675,...,civic_institutional,healthcare,commercial,retail,20,23f-222@62j-sjv-b6k,42.372895,-71.117919,42.375417,-71.10716
4,1.46072351e+19,EAT,MAINTENANCE,2023-02-03 13:09:58,2023-02-03 13:29:30,19,1.01223210,NaN,250173537001,42.3750675,...,civic_institutional,healthcare,commercial,retail,20,237-222@62j-sj3-b8v,42.379450,-71.096153,42.375417,-71.10716


In [257]:
# Save the relevant data for the GAMA simulation model
healthcare_trips_df=replica_healthcare_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                         'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
healthcare_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,9.47840871e+17,42.368397,-71.116023,42.374964,-71.108469,EAT,2023-02-03 10:19:03,2023-02-03 10:24:11,5,1.48284668,20
1,1.25025580e+19,42.372746,-71.099375,42.375417,-71.10716,EAT,2023-02-03 10:28:52,2023-02-03 10:30:47,1,0.56743335,20
2,7.85785543e+18,42.374052,-71.100069,42.374964,-71.108469,EAT,2023-02-03 11:17:24,2023-02-03 11:19:51,2,0.57928027,20
3,1.37671630e+19,42.372895,-71.117919,42.375417,-71.10716,EAT,2023-02-03 12:12:08,2023-02-03 12:15:18,3,0.87821869,20
4,1.46072351e+19,42.379450,-71.096153,42.375417,-71.10716,EAT,2023-02-03 13:09:58,2023-02-03 13:29:30,19,1.01223210,20


# MERGE

In [258]:
# Merge all the trips to generate the final eat trips dataset
final_eattrips_df = pd.concat([residential_trips_df,industrial_trips_df,office_trips_df,shopping_trips_df,hotel_trips_df,restaurant_trips_df,work_trips_df,non_retail_trips_df,park_trips_df,transportationutilities_trips_df,civic_institutional_trips_df,education_trips_df,healthcare_trips_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
final_eattrips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.22674427e+19,42.367673,-71.090329,42.373454,-71.089322,EAT,2023-02-03 00:03:19,2023-02-03 00:05:28,2,0.11179820,20
1,1.17350550e+19,42.377946,-71.107261,42.373983,-71.107175,EAT,2023-02-03 00:08:15,2023-02-03 00:09:07,0,0.04538256,20
2,1.76322124e+19,42.365028,-71.102423,42.377015,-71.092825,EAT,2023-02-03 00:12:03,2023-02-03 00:31:47,19,1.02195261,20
3,1.38419628e+19,42.363664,-71.101451,42.375492,-71.110654,EAT,2023-02-03 00:13:11,2023-02-03 00:36:32,23,1.20961137,20
4,1.62290058e+19,42.371281,-71.083318,42.372122,-71.087128,EAT,2023-02-03 00:34:13,2023-02-03 00:34:32,0,0.01652482,20


In [259]:
len(final_eattrips_df)

7743

# CONNECT TRIPS AND SPEND

In [260]:
# Import the spend data from Safegraph
filepath_Replica_Spend = ".././02 Data/Replica/Spend/Tangible-Table/"
replica_spend_inperson_df = pd.read_csv(filepath_Replica_Spend + 'tangibletable_restaurants_and_bars_weekof_01152022_to_01212023_in-person_spend.csv')
replica_spend_inperson_df = replica_spend_inperson_df.rename(columns={'count': 'count_inperson'}, errors='raise')
replica_spend_inperson_df = replica_spend_inperson_df.rename(columns={'change_from_baseline': 'change_from_baseline_inperson'}, errors='raise')
replica_spend_online_df = pd.read_csv(filepath_Replica_Spend + 'tangibletable_restaurants_and_bars_weekof_01152022_to_01212023_online_spend.csv')
replica_spend_online_df = replica_spend_online_df.rename(columns={'count': 'count_online'}, errors='raise')
replica_spend_online_df = replica_spend_online_df.rename(columns={'change_from_baseline': 'change_from_baseline_online'}, errors='raise')

In [261]:
# Merge both spend files in the same one
replica_spend_df = replica_spend_inperson_df.merge(replica_spend_online_df[['id','count_online', 'change_from_baseline_online']], how='left', left_on='id', right_on='id')
replica_spend_df.head()

,id,id_label,metric,count_inperson,change_from_baseline_inperson,count_online,change_from_baseline_online
0,Week:202203,"Jan 15, 2022",EstimatedSpendHome,5130918.5263,0.00000000,1377366.8715,0.00000000
1,Week:202204,"Jan 22, 2022",EstimatedSpendHome,5336889.5624,0.04014311,1940579.2088,0.40890510
2,Week:202205,"Jan 29, 2022",EstimatedSpendHome,3997615.5289,-0.22087722,1179159.2455,-0.14390329
3,Week:202206,"Feb 5, 2022",EstimatedSpendHome,4890578.8259,-0.04684146,1488689.6737,0.08082291
4,Week:202207,"Feb 12, 2022",EstimatedSpendHome,5378095.9475,0.04817411,1736613.9934,0.26082167


In [262]:
# Calculate percentage of the money spend online comparing to the total
replica_spend_df['%online'] = replica_spend_df['count_online']/(replica_spend_df['count_online']+replica_spend_df['count_inperson'])*100
replica_spend_df.head()

,id,id_label,metric,count_inperson,change_from_baseline_inperson,count_online,change_from_baseline_online,%online
0,Week:202203,"Jan 15, 2022",EstimatedSpendHome,5130918.5263,0.00000000,1377366.8715,0.00000000,21.16328322
1,Week:202204,"Jan 22, 2022",EstimatedSpendHome,5336889.5624,0.04014311,1940579.2088,0.40890510,26.66557934
2,Week:202205,"Jan 29, 2022",EstimatedSpendHome,3997615.5289,-0.22087722,1179159.2455,-0.14390329,22.77787420
3,Week:202206,"Feb 5, 2022",EstimatedSpendHome,4890578.8259,-0.04684146,1488689.6737,0.08082291,23.33636958
4,Week:202207,"Feb 12, 2022",EstimatedSpendHome,5378095.9475,0.04817411,1736613.9934,0.26082167,24.40878135


In [263]:
# Calculate de average percentage of online spend
average = replica_spend_df['%online'].mean()
average

19.90227893196388

In [264]:
# Definition of inperson vs first iteration of online trips
in_person_trips = len(eattrips_df) + l_nothing
online_trips = in_person_trips*average/(100-average)

In [265]:
# Comparison of the trips needed to do online vs in person depending on the order size
avg_online_order = 33.26
avg_inperson_order = 60.40
travels = avg_inperson_order/avg_online_order
# Calculation of real online trips
online_trips = round(online_trips*travels)
# Calculation of the trips to delate
delete_trips = in_person_trips-online_trips

In [266]:
online_trips

3604

In [267]:
l_nothing

243

In [268]:
delete_trips

4383

In [269]:
final_eattrips_df['same_place']=' '
count_same = 0
for i in range(len(final_eattrips_df)):
    if (final_eattrips_df.at[i,'start_latitude']==final_eattrips_df.at[i,'end_latitude'])&(final_eattrips_df.at[i,'start_longitude']==final_eattrips_df.at[i,'end_longitude']):
        final_eattrips_df.at[i,'same_place']='Yes'
        count_same = count_same + 1
    else:
        final_eattrips_df.at[i,'same_place']='No'

final_eattrips_df=final_eattrips_df[final_eattrips_df['same_place']=='No']
final_eattrips_df=final_eattrips_df.reset_index().drop('index',axis=1)
final_eattrips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour,same_place
0,1.22674427e+19,42.367673,-71.090329,42.373454,-71.089322,EAT,2023-02-03 00:03:19,2023-02-03 00:05:28,2,0.11179820,20,No
1,1.17350550e+19,42.377946,-71.107261,42.373983,-71.107175,EAT,2023-02-03 00:08:15,2023-02-03 00:09:07,0,0.04538256,20,No
2,1.76322124e+19,42.365028,-71.102423,42.377015,-71.092825,EAT,2023-02-03 00:12:03,2023-02-03 00:31:47,19,1.02195261,20,No
3,1.38419628e+19,42.363664,-71.101451,42.375492,-71.110654,EAT,2023-02-03 00:13:11,2023-02-03 00:36:32,23,1.20961137,20,No
4,1.62290058e+19,42.371281,-71.083318,42.372122,-71.087128,EAT,2023-02-03 00:34:13,2023-02-03 00:34:32,0,0.01652482,20,No


In [270]:
final_eattrips_df_copy = final_eattrips_df.copy()
count_distance_zero = final_eattrips_df[final_eattrips_df['duration_minutes']==0].shape[0]
final_eattrips_df = final_eattrips_df[(final_eattrips_df['duration_minutes']==0)==False]
final_eattrips_df = final_eattrips_df.reset_index().drop('index',axis=1)
len(final_eattrips_df)

6516

In [271]:
# Same of the trips that need to be deleted have already been deleted in steps before
extra_delete_trips = delete_trips-l_nothing-count_same-count_distance_zero-1

In [272]:
count_distance_zero

1175

In [273]:
# Delete random trips and generate the online delivery dataset
from numpy.random import default_rng

trips_drop = default_rng().choice(final_eattrips_df.index, size=extra_delete_trips, replace=False)
onlinetrips_df=final_eattrips_df.drop(index=trips_drop).reset_index().drop('index',axis=1)

In [274]:
# Change format of start_time and end_time to be able to read in GAMA
onlinetrips_df['start_time']=pd.to_datetime(onlinetrips_df['start_time'])
onlinetrips_df['end_time']=pd.to_datetime(onlinetrips_df['end_time'])
onlinetrips_df

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour,same_place
0,1.38419628e+19,42.363664,-71.101451,42.375492,-71.110654,EAT,2023-02-03 00:13:11,2023-02-03 00:36:32,23,1.20961137,20,No
1,2.74441125e+18,42.361745,-71.097077,42.364506,-71.097976,EAT,2023-02-03 00:47:20,2023-02-03 00:52:15,4,0.25483028,20,No
2,1.59769228e+19,42.372208,-71.115966,42.374182,-71.116059,EAT,2023-02-03 00:59:15,2023-02-03 01:00:58,1,0.73943068,20,No
3,4.39099190e+18,42.377298,-71.104867,42.370917,-71.114905,EAT,2023-02-03 00:59:15,2023-02-03 01:02:28,3,1.38018328,20,No
4,1.77660326e+19,42.378182,-71.106010,42.371219,-71.117438,EAT,2023-02-03 00:59:15,2023-02-03 01:22:06,22,1.18365263,20,No
...,...,...,...,...,...,...,...,...,...,...,...,...
3599,1.09989641e+17,42.372775,-71.099329,42.372992,-71.104019,EAT,2023-02-03 23:18:36,2023-02-03 23:20:02,1,0.58653128,20,No
3600,1.83562425e+19,42.374155,-71.101091,42.374521,-71.101502,EAT,2023-02-03 23:19:11,2023-02-03 23:20:52,1,0.08754172,20,No
3601,1.77516130e+19,42.363335,-71.100716,42.362183,-71.100137,EAT,2023-02-03 23:21:13,2023-02-03 23:22:49,1,0.08327946,20,No
3602,1.20320094e+17,42.369072,-71.100965,42.366359,-71.101454,EAT,2023-02-03 23:29:46,2023-02-03 23:30:53,1,0.38245059,20,No


In [275]:
# Save the results of final food delivery trips
results_path=Path('./Input Database/fooddeliverytrips_tangibletable.csv')
onlinetrips_df.to_csv(results_path,index=False)